In [1]:
import math

In [2]:
import pandas as pd

import numpy as np

import nltk

import re
from collections import Counter

import string

import pprint

In [3]:
from statistics import mean

In [4]:
from itertools import islice

In [5]:
import scipy

Nb Formatting 

In [6]:
pd.options.display.max_seq_items = 2000

pd.set_option('display.max_columns', 100)

# Data setup

Open Dataset 

In [7]:
import os

In [8]:
os.getcwd()

'/Users/renee/Documents/Spec'

In [9]:
#ignore header for easier (ordinal) indexing
datasheet = pd.read_excel('CogData_FU_82818.xlsx',header=None)

In [10]:
len(datasheet)

577

In [11]:
datasheet.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96
0,Record ID,Event Name,Write a short sketch about a memory from your ...,How does technology and social media impact th...,Date of Neurobehavioral Status Exam,Notes,NIH Toolbox ID,NIH Toolbox Password,Crystallized Composite fully adjusted,Picture Vocab fully adjusted,Picture Vocab age adjusted,Picture Vocab computed,Oral Reading Recognition fully adjusted,Oral Reading Recognition age adjusted,Oral Reading Recognition computed,RAVLT Trial1,RAVLT Trial2,RAVLT Trial3,RAVLT Delayed Recall,RAVLT: Total Recognition Score,RAVLT: False Positive Score,RAVLT Percentile,Picture Sequence Memory fully adjusted,Picture Sequence Age Adjusted,Picture Sequence Memory computed score,List Sorting fully adjusted,List Sorting Age Adjusted,List Sorting raw score,Dimensional Change Card Sort Fully Adjusted,Dimensional Change Age Adjusted,Dimensional Change computed score,Dimensional Change raw score,Flanker Inhibitory Attention & Control Fully A...,Flanker Inhibitory Attention & Control age adj...,Flanker computed score,Flanker raw score,Pattern Comparison Fully Adjusted Scale Score,Pattern Comparison age adjusted,Pattern Comparison Raw Score,Odor Identification Fully Adjusted Scale Score,Odor Identification age adjusted,Odor Identification (Raw),Oral Digit Symbol Percentile,Oral Digit Symbol (Raw),Date of Short Cognitive Testing,FAS Letter Fluency,FAS Percentile,FAS Fully Adjusted,Animal Naming Test,Animal Naming Test Percentile,Animal Naming Fully Adjusted,Trail Making Test B,Trail Making Test B Percentile,Trail Making Fully Adjusted,Boston Naming,Boston Naming Percentile,Boston Naming Fully Adjusted,Neurotrack ID,Neurotrack Date,NT Mean Novelty Preference Score,Neurotrack Score (Median),1. Orientation Score,1a. Time,1b. Place,2. Registration,3. Attention and Calculation,4. Recall,5. Language,5a. Naming,5b. Repetition,5c. 3-stage Command,5d. Reading,5e. Writing,5f. Copying,Total MMSE score,Story Version,Immediate Recall,Immediate Recall Percentile,Delayed Recall,Delayed Recall Percentile,Facial Recognition A,Cued First Letter A,Multiple Choice Names A,Facial Recognition B,Cued First Letter B,Multiple Choice Names B,CogStateID,Date Conducted:,CogState: Detection,CogState: Identification,CogState: One Card Learning,CogState: One Back Speed,Cog State: One Back Accuracy,Ravens Progressive Matrices,Logical Memory Part B,East Boston Immediate Recall,Complete?
1,10,Baseline,NaN,NaN,2015-02-23 00:00:00,"RAVLT recognition, Odor ID and Trails B not ad...",85176,NaN,83.32,73.6571,83.1442,1464,100.106,109.371,2180,2,3,3,0,NaN,NaN,<25,95.04,95.98,407.08,68.03,74.62,8,82.9,88.57,5.85,30,68.51,72.43,4.88,19,67.64,70.46,28,NaN,NaN,NaN,<min,26,NaN,21,<10,NaN,7,<10,NaN,NaN,NaN,NaN,12,NaN,NaN,NaN,NaN,NaN,NaN,5,1,4,3,4,0,9,2,1,3,1,1,1,21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Complete
2,10,FU Visit 2,NaN,NaN,2015-09-28 00:00:00,Trails B incomplete,65967,NaN,129.69,104.74,107.1,1739,127.399,132.224,2578,3,3,4,0,9,16,<25,NaN,NaN,NaN,NaN,NaN,NaN,81.81,87.53,5.74,25,81.03,86.93,6.85,20,54.96,58.74,18,91.83,89.99,6,<25,35,NaN,27,20-30,NaN,10,<10,NaN,NaN,NaN,NaN,11,NaN,NaN,S_172,2015-09-28 00:00:00,0.548938,0.642857,5,2,3,3,5,0,9,2,1,3,1,1,1,22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Complete
3,13,Baseline,NaN,NaN,2015-02-24 00:00:00,Odor ID not administered,15497,NaN,140.09,122.101,126.974,2001,153.735,153.95,5220,8,10,14,12,NaN,NaN,>75,113.71,114.05,596.02,122.3,128,23,101.5,108.28,8.65,30,83.14,87.7,7.9,20,114.81,118.54,67,NaN,NaN,NaN,>75,108,NaN,70,>90,NaN,35,>90,NaN,NaN,NaN,NaN,14,NaN,NaN,NaN,NaN,NaN,NaN,10,5,5,3,5,3,9,2,1,3,1,1,1,30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,401201572678100,2015-04-01 

Row Indices of Memory Responses 

In [12]:
#ignore first (header) column
memory_ix = datasheet.iloc[1:][datasheet.iloc[1:,2].notnull()].index


Record IDs of Memory Respondents [Analyze all scores from each participant]

In [13]:
memory_id = list(datasheet.iloc[memory_ix][0])

Confirm Unique Response-Respondents 

In [14]:
len(memory_id)

46

In [15]:
len(set(memory_id))

46

In [16]:
datasheet.iloc[memory_ix]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96
6,14,Baseline,When I was a little child I had the same exper...,I stay in touch with family and friends throug...,2015-02-26 00:00:00,NaN,35926,qjh,123.54,126.545,131.008,2048,114.984,120.673,2378,4,3,6,2,NaN,NaN,NaN,99.13,104.48,455.07,94.85,101.4,14,106.02,112.97,7.52,30,84.37,88.37,6.97,20,97.93,102.12,47,NaN,NaN,NaN,25-50,57,NaN,46,60-70,NaN,20,50-75,NaN,NaN,NaN,NaN,15,NaN,NaN,NaN,NaN,NaN,NaN,10,5,5,3,5,3,9,2,1,3,1,1,1,30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,317201572669916,2015-03-17 00:00:00,102,99,103,93,116,NaN,NaN,NaN,Complete
29,26,Baseline,"When I was very young, living in the Astoria n...",I'm a journalist who has covered the rise of t...,2015-02-03 00:00:00,RAVLT rec not administered,36080,NaN,148.32,145.587,151.803,2387,123.836,129.301,2536,5,9,12,3,NaN,NaN,>75,98.49,99.03,441.15,117.08,123.29,20,90.58,96.61,7.23,30,80.58,84.76,7.13,20,96.19,99.72,50,NaN,NaN,NaN,<25,56,NaN,53,80,NaN,26,>90,NaN,NaN,NaN,NaN,15,NaN,NaN,NaN,NaN,NaN,NaN,10,5,5,3,5,3,9,2,1,3,1,1,1,30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1054,2015-02-03 00:00:00,113,112,100,100,103,NaN,NaN,NaN,Complete
51,64,Baseline,I went to Paris when I was 16. I was staying ...,My husband and I are on our computers all day ...,2015-05-08 00:00:00,No Rey recognition,61681,NaN,130.68,134.57,139.464,2163,115.002,120.701,2379,7,10,11,6,NaN,NaN,>75,NaN,NaN,NaN,NaN,NaN,NaN,116.76,122.84,8.3,30,113.97,119.91,8.68,20,122.25,126.13,60,132.15,137.68,9,50-75,62,NaN,47,60-70,NaN,22,75,NaN,84,50,NaN,15,NaN,NaN,S_53,2015-05-08 00:00:00,poor quality,NaN,10,5,5,3,5,3,9,2,1,3,1,1,1,30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,61620151847168,2015-06-16 00:00:00,NaN,NaN,80,96,61,NaN,NaN,NaN,Complete
56,67,Baseline,My husband and I had always wanted to go to Pa...,"I'm on the computer all the time, writing or r...",2015-03-17 00:00:00,No RAVLT recognition and Odor ID,56478,NaN,130.99,129.869,132.736,2131,120.448,123.786,2449,7,10,12,10,NaN,NaN,>75,NaN,NaN,NaN,NaN,NaN,NaN,80.55,83.58,6.04,29,82.62,86.65,7.28,20,72,75.96,36,NaN,NaN,NaN,25-50,62,2015-03-17 00:00:00,49,70,NaN,29,>90,NaN,72,20-30,NaN,14,NaN,NaN,I_11,2015-03-17 00:00:00,0.570714,0.6,10,5,5,3,5,3,9,1,1,3,1,1,1,30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,325201574035660,2015-03-25 00:00:00,96,105,109,99,103,NaN,NaN,NaN,Complete
78,101,Baseline,I was born in Hungary. A significant memory is...,My use of social media is minimal. I only use ...,2015-03-06 00:00:00,No RAVLT rec and Odor ID,52303,NaN,123.55,117.09,121.136,1967,120.986,124.254,2455,6,9,10,7,NaN,NaN,>75,NaN,NaN,NaN,97.45,103.11,16,92.3,97.7,7.31,29,78.6,81.54,6.8,20,88.52,92.83,46,NaN,NaN,NaN,25-50,60,NaN,30,2018-10-20 00:00:00,NaN,11,<10,NaN,66,60-70,NaN,13,NaN,NaN,I_00,2015-03-06 00:00:00,0.470714,0.5,10,5,5,3,5,2,9,2,1,3,1,1,1,29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,422201573922344,2015-04-22 00:00:00,NaN,113,72,NaN,NaN,NaN,NaN,NaN,Complete
89,110,Baseline,When I was 16 years old I traveled to Europe b...,I check that on what my children are doing and...,2016-03-04 00:00:00,NaN,78039,vab,137.14,133,137.148,2164,124.176,129.631,2505,9,12,15,13,14,1,>75,NaN,NaN,NaN,NaN,NaN,NaN,111.59,118.1,9,30,118.52,124.95,9.56,20,128.01,132.7,70,111.87,118.02,9,50-75,78,2016-03-04 00:00:00,76,>90,NaN,28,75-90,NaN,35,>90,NaN,15,NaN,NaN,I_338,2016-03-01 00:00:00,0.595952,0.595952,10,5,5,3,5,3,9,2,1,3,1,1,1,30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,304201673920436,2016-03-04 00:00:00,106,105,112,102,103,NaN,NaN,NaN,Complete
100,135,Baseline,I have great memories of growing up in huge ho...,Technology has helped me connect to family and...,2015-04-20 00:00:00,No RAVLT rec.,59106,NaN,105.68,106.366,108.008,1767,104.435,109.664,2163,4,8,9,7,NaN,NaN,25-50,NaN,NaN,NaN,NaN,NaN,NaN,

Tech respondents

In [17]:
tech_ix = datasheet.iloc[1:][datasheet.iloc[1:,3].notnull()].index

In [18]:
tech_ix

Int64Index([  6,  29,  51,  56,  78,  89, 100, 107, 118, 154, 184, 202, 217,
            226, 234, 242, 266, 277, 280, 286, 297, 303, 324, 337, 347, 352,
            359, 375, 387, 398, 411, 423, 433, 438, 445, 448, 460, 464, 470,
            474, 477, 496, 499, 502, 506, 509, 519, 522, 537, 540],
           dtype='int64')

In [19]:
tech_id = list(datasheet.iloc[tech_ix][0])

In [20]:
len(tech_id)

50

In [21]:
#each respondent responded to one prompt
len(tech_id) == len(set(tech_id))

True

In [22]:
ix_id_series = pd.Series(tech_id,index=tech_ix)

In [23]:
ix_id_series

6       14
29      26
51      64
56      67
78     101
89     110
100    135
107    139
118    145
154    205
184    233
202    245
217    254
226    263
234    271
242    276
266    291
277    310
280    320
286    322
297    331
303    342
324    382
337    403
347    417
352    420
359    434
375    458
387    465
398    474
411    483
423    512
433    514
438    521
445    532
448    537
460    559
464    567
470    579
474    583
477    587
496    602
499    603
502    604
506    607
509    609
519    616
522    619
537    626
540    631
dtype: int64

Tech-exclusive respondents

In [24]:
#row indices of tech-respondents, mem-negligent
tech_x_ix = [tix for tix in tech_ix if tix not in memory_ix]

In [25]:
#record ids of tech-respondents who are mem-negligent
tech_x_ids = [tid for tid in tech_id if tid not in memory_id]

Mem-exclusive respondents

In [26]:
#mem respondents who are tech-negligent
[m for m in memory_id if m not in tech_id]

[]

Indices of all Respondent-rows, by record ID

In [27]:
#all rows for tech_respondents
tech_all_ix = [ix for ix in datasheet.index if datasheet.iloc[ix,0] in tech_id]

In [28]:
max(datasheet[datasheet[0]==14].index)

12

In [29]:
datasheet.iloc[tech_all_ix]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96
6,14,Baseline,When I was a little child I had the same exper...,I stay in touch with family and friends throug...,2015-02-26 00:00:00,NaN,35926,qjh,123.54,126.545,131.008,2048,114.984,120.673,2378,4,3,6,2,NaN,NaN,NaN,99.13,104.48,455.07,94.85,101.4,14,106.02,112.97,7.52,30,84.37,88.37,6.97,20,97.93,102.12,47,NaN,NaN,NaN,25-50,57,NaN,46,60-70,NaN,20,50-75,NaN,NaN,NaN,NaN,15,NaN,NaN,NaN,NaN,NaN,NaN,10,5,5,3,5,3,9,2,1,3,1,1,1,30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,317201572669916,2015-03-17 00:00:00,102,99,103,93,116,NaN,NaN,NaN,Complete
7,14,FU Visit 2,NaN,NaN,2015-07-14 00:00:00,NaN,82858,NaN,126.32,124.077,129.483,2018,119.487,124.812,2448,4,8,9,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,122.73,127.93,8.87,27,113.43,119.36,8.65,20,122.25,126.13,60,77.81,80.25,5,<50,59,NaN,50,>70,NaN,16,25,NaN,90,30-40,NaN,15,NaN,NaN,NaN,NaN,NaN,NaN,10,5,5,3,5,3,9,2,1,3,1,1,1,30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,702201572669916,2015-07-02 00:00:00,108,99,95,91,107,NaN,NaN,NaN,Complete
8,14,FU Visit 3,NaN,NaN,2016-03-07 00:00:00,NaN,48846,evf,132.09,132.171,135.169,2128,119.993,125.099,2457,2,7,8,3,9,1,50-75,NaN,NaN,NaN,NaN,NaN,NaN,125.63,131.41,9.22,30,116.99,122.7,8.84,20,122.25,126.13,60,112,116.35,8,50-75,69,2016-03-07 00:00:00,50,80-90,NaN,25,>90,NaN,95,60,NaN,15,NaN,NaN,I_340,2016-03-07 00:00:00,unscorable,NaN,9,4,5,3,5,3,9,2,1,3,1,1,1,29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,307201672669916,2016-03-07 00:00:00,102,98,110,92,101,NaN,NaN,NaN,Complete
9,14,FU Visit 4,NaN,NaN,2016-08-25 00:00:00,NaN,94255,yua,126.02,132.98,136.123,2138,113.024,118.634,2344,7,9,11,9,11,0,>75,NaN,NaN,NaN,NaN,NaN,NaN,120.66,125.95,8.66,29,110.39,116.18,8.5,20,112.71,117.19,55,87.26,89.99,6,50,60,2016-08-25 00:00:00,45,70-80,NaN,16,25-50,NaN,95,50,NaN,15,NaN,NaN,I_410,2016-08-25 00:00:00,unscorable,NaN,10,5,5,3,5,3,9,2,1,3,1,1,1,30,A,10,NaN,8,NaN,12,1,8,NaN,NaN,NaN,702201572669916,2016-08-25 00:00:00,105,94,104,86,103,NaN,NaN,NaN,Complete
10,14,FU Visit 5,NaN,NaN,2017-02-17 00:00:00,NaN,45760,mur,134.39,134.534,139.433,2163,119.738,124.89,2452,5,6,8,5,11,2,50-75,NaN,NaN,NaN,NaN,NaN,NaN,116.47,122.56,8.27,29,116.44,122.32,8.8,20,116.53,120.58,57,98.2,101.73,7,50-75,71,2017-02-17 00:00:00,55,80-90,NaN,24,75-90,NaN,103,30,NaN,15,NaN,NaN,I_481,2017-02-17 00:00:00,NaN,NaN,10,5,5,3,5,3,9,2,1,3,1,1,1,30,B,8,NaN,6,NaN,12,3,5,NaN,NaN,NaN,217201772669916,2017-02-17 00:00:00,104,98,106,93,101,NaN,NaN,NaN,Complete
11,14,FU Visit 6,NaN,NaN,2017-09-15 00:00:00,NaN,84695,hfa,120.9,130.529,134.17,2111,105.981,113.037,2241,4,6,9,4,11,6,50-75,NaN,NaN,NaN,NaN,NaN,NaN,118.24,124.23,8.42,30,113.82,119.68,8.67,20,120.2,124.32,59,98.2,101.73,7,50-75,63,2017-09-15 00:00:00,40,40-50,NaN,18,25-50,NaN,84,50,NaN,15,NaN,NaN,I_597,2017-09-15 00:00:00,unscorable,NaN,10,5,5,3,5,3,9,2,1,3,1,1,1,30,A,11,NaN,11,NaN,NaN,NaN,NaN,12,5,11,217201772669916,2017-09-15 00:00:00,102,98,105,93,103,NaN,NaN,NaN,Complete
12,14,FU Visit 7,NaN,NaN,2018-04-16 00:00:00,NaN,12836,wtc,132.08,135.857,141.764,2189,114.399,120.015,2370,5,7,10,6,10,1,>75,NaN,NaN,NaN,NaN,NaN,NaN,119.76,125.35,8.56,30,108.67,114.54,8.41,20,125.84,129.77,62,98.2,101.73,7,50-75,61,2018-04-16 00:00:00,51,70-80,NaN,17,25-50,NaN,113,<10,NaN,15,NaN,NaN,NaN,NaN,NaN,NaN,9,4,5,3,5,2,9,2,1,3,1,1,1,28,B,10,NaN,9,NaN,12,5,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Complete
29,26,Baseline,"When I was very young, living in the Astoria n...",I'm a journalist who has covered the rise of t...,2015-02-03 00:00:00,RAVLT rec not administered,36080,NaN,148.32,145.587,151.803,2387,123.836,129.301,2536,5,9,12,3,NaN,NaN,>75,98.49,99.03,441.15,117.08,123.29,20,90.58,96.61,7.23,30,80.58,84.76,7.13,20,96.19,99.72,50,NaN,NaN,NaN,<25,56,NaN,53,80,NaN,26,>90,NaN

In [30]:
mem_all_ix = [ix for ix in datasheet.index if datasheet.iloc[ix,0] in memory_id]

In [31]:
datasheet.iloc[mem_all_ix]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96
6,14,Baseline,When I was a little child I had the same exper...,I stay in touch with family and friends throug...,2015-02-26 00:00:00,NaN,35926,qjh,123.54,126.545,131.008,2048,114.984,120.673,2378,4,3,6,2,NaN,NaN,NaN,99.13,104.48,455.07,94.85,101.4,14,106.02,112.97,7.52,30,84.37,88.37,6.97,20,97.93,102.12,47,NaN,NaN,NaN,25-50,57,NaN,46,60-70,NaN,20,50-75,NaN,NaN,NaN,NaN,15,NaN,NaN,NaN,NaN,NaN,NaN,10,5,5,3,5,3,9,2,1,3,1,1,1,30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,317201572669916,2015-03-17 00:00:00,102,99,103,93,116,NaN,NaN,NaN,Complete
7,14,FU Visit 2,NaN,NaN,2015-07-14 00:00:00,NaN,82858,NaN,126.32,124.077,129.483,2018,119.487,124.812,2448,4,8,9,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,122.73,127.93,8.87,27,113.43,119.36,8.65,20,122.25,126.13,60,77.81,80.25,5,<50,59,NaN,50,>70,NaN,16,25,NaN,90,30-40,NaN,15,NaN,NaN,NaN,NaN,NaN,NaN,10,5,5,3,5,3,9,2,1,3,1,1,1,30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,702201572669916,2015-07-02 00:00:00,108,99,95,91,107,NaN,NaN,NaN,Complete
8,14,FU Visit 3,NaN,NaN,2016-03-07 00:00:00,NaN,48846,evf,132.09,132.171,135.169,2128,119.993,125.099,2457,2,7,8,3,9,1,50-75,NaN,NaN,NaN,NaN,NaN,NaN,125.63,131.41,9.22,30,116.99,122.7,8.84,20,122.25,126.13,60,112,116.35,8,50-75,69,2016-03-07 00:00:00,50,80-90,NaN,25,>90,NaN,95,60,NaN,15,NaN,NaN,I_340,2016-03-07 00:00:00,unscorable,NaN,9,4,5,3,5,3,9,2,1,3,1,1,1,29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,307201672669916,2016-03-07 00:00:00,102,98,110,92,101,NaN,NaN,NaN,Complete
9,14,FU Visit 4,NaN,NaN,2016-08-25 00:00:00,NaN,94255,yua,126.02,132.98,136.123,2138,113.024,118.634,2344,7,9,11,9,11,0,>75,NaN,NaN,NaN,NaN,NaN,NaN,120.66,125.95,8.66,29,110.39,116.18,8.5,20,112.71,117.19,55,87.26,89.99,6,50,60,2016-08-25 00:00:00,45,70-80,NaN,16,25-50,NaN,95,50,NaN,15,NaN,NaN,I_410,2016-08-25 00:00:00,unscorable,NaN,10,5,5,3,5,3,9,2,1,3,1,1,1,30,A,10,NaN,8,NaN,12,1,8,NaN,NaN,NaN,702201572669916,2016-08-25 00:00:00,105,94,104,86,103,NaN,NaN,NaN,Complete
10,14,FU Visit 5,NaN,NaN,2017-02-17 00:00:00,NaN,45760,mur,134.39,134.534,139.433,2163,119.738,124.89,2452,5,6,8,5,11,2,50-75,NaN,NaN,NaN,NaN,NaN,NaN,116.47,122.56,8.27,29,116.44,122.32,8.8,20,116.53,120.58,57,98.2,101.73,7,50-75,71,2017-02-17 00:00:00,55,80-90,NaN,24,75-90,NaN,103,30,NaN,15,NaN,NaN,I_481,2017-02-17 00:00:00,NaN,NaN,10,5,5,3,5,3,9,2,1,3,1,1,1,30,B,8,NaN,6,NaN,12,3,5,NaN,NaN,NaN,217201772669916,2017-02-17 00:00:00,104,98,106,93,101,NaN,NaN,NaN,Complete
11,14,FU Visit 6,NaN,NaN,2017-09-15 00:00:00,NaN,84695,hfa,120.9,130.529,134.17,2111,105.981,113.037,2241,4,6,9,4,11,6,50-75,NaN,NaN,NaN,NaN,NaN,NaN,118.24,124.23,8.42,30,113.82,119.68,8.67,20,120.2,124.32,59,98.2,101.73,7,50-75,63,2017-09-15 00:00:00,40,40-50,NaN,18,25-50,NaN,84,50,NaN,15,NaN,NaN,I_597,2017-09-15 00:00:00,unscorable,NaN,10,5,5,3,5,3,9,2,1,3,1,1,1,30,A,11,NaN,11,NaN,NaN,NaN,NaN,12,5,11,217201772669916,2017-09-15 00:00:00,102,98,105,93,103,NaN,NaN,NaN,Complete
12,14,FU Visit 7,NaN,NaN,2018-04-16 00:00:00,NaN,12836,wtc,132.08,135.857,141.764,2189,114.399,120.015,2370,5,7,10,6,10,1,>75,NaN,NaN,NaN,NaN,NaN,NaN,119.76,125.35,8.56,30,108.67,114.54,8.41,20,125.84,129.77,62,98.2,101.73,7,50-75,61,2018-04-16 00:00:00,51,70-80,NaN,17,25-50,NaN,113,<10,NaN,15,NaN,NaN,NaN,NaN,NaN,NaN,9,4,5,3,5,2,9,2,1,3,1,1,1,28,B,10,NaN,9,NaN,12,5,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Complete
29,26,Baseline,"When I was very young, living in the Astoria n...",I'm a journalist who has covered the rise of t...,2015-02-03 00:00:00,RAVLT rec not administered,36080,NaN,148.32,145.587,151.803,2387,123.836,129.301,2536,5,9,12,3,NaN,NaN,>75,98.49,99.03,441.15,117.08,123.29,20,90.58,96.61,7.23,30,80.58,84.76,7.13,20,96.19,99.72,50,NaN,NaN,NaN,<25,56,NaN,53,80,NaN,26,>90,NaN

Column index Dict

Investigate score spread

In [32]:
datasheet.iloc[tech_ix,51:54]

,51,52,53
6,NaN,NaN,NaN
29,NaN,NaN,NaN
51,84,50,NaN
56,72,20-30,NaN
78,66,60-70,NaN
89,35,>90,NaN
100,74,20-30,NaN
107,NaN,NaN,NaN
118,55,80-90,NaN
154,45,80-90,NaN


In [33]:
#MMSE
datasheet.iloc[tech_all_ix][[0,1,4,74]]

,0,1,4,74
6,14,Baseline,2015-02-26 00:00:00,30
7,14,FU Visit 2,2015-07-14 00:00:00,30
8,14,FU Visit 3,2016-03-07 00:00:00,29
9,14,FU Visit 4,2016-08-25 00:00:00,30
10,14,FU Visit 5,2017-02-17 00:00:00,30
11,14,FU Visit 6,2017-09-15 00:00:00,30
12,14,FU Visit 7,2018-04-16 00:00:00,28
29,26,Baseline,2015-02-03 00:00:00,30
30,26,FU Visit 2,2015-07-28 00:00:00,30
31,26,FU Visit 3,2016-01-28 00:00:00,29


In [34]:
#MMSE scores
#6 unique...
np.unique([int(i) for i in datasheet.iloc[tech_all_ix][74].dropna()],return_counts=True)

(array([23, 26, 27, 28, 29, 30]), array([  1,   2,   1,  16,  48, 152]))

In [35]:
#Min MMSE
datasheet.iloc[tech_all_ix][74].min()

23

In [36]:
#The maximum MMSE score is 30 points. A score of 20 to 24 suggests mild dementia, 13 to 20 suggests moderate dementia, and less than 12 indicates severe dementia.
#participants with MCI...one ---
datasheet.iloc[tech_all_ix][74][datasheet.iloc[tech_all_ix][74] < 25]

390    23
Name: 74, dtype: object

In [37]:
#Unique words

In [38]:
len(tech_ix)

50

## Lancet dataset

In [39]:
lancet = pd.read_excel('Lancet_82318.xlsx',header=None)

In [40]:
[(i,name) for (i,name) in enumerate(lancet.iloc[0])]

[(0, 'Record ID'),
 (1, 'Event Name'),
 (2, 'Gender'),
 (3, 'Reason for appointment'),
 (4, 'Other'),
 (5, 'Calculated Age'),
 (6,
  "Has someone in your family ever been diagnosed with Alzheimer's Disease?"),
 (7,
  'Were you or anyone in your close family ever diagnosed by an education specialist with a learning disability or ADHD?'),
 (8, 'Are you having any symptoms of memory loss or cognitive impairment?'),
 (9,
  'Do you think the patient could undertake one hour of cognitive testing?'),
 (10, 'Physician'),
 (11,
  'Are you coming from out of town, or do you have a lengthy travel time to the clinic?'),
 (12, 'Type of visit?'),
 (13, 'Date of first visit?'),
 (14, 'Date of Consent'),
 (15, 'Complete?'),
 (16, 'Clinical Diagnosis (RSI)'),
 (17, 'Clinical Diagnosis (RSI_Abbreviated) '),
 (18, 'Doctor-Rated Compliance (0-10)'),
 (19, 'Date of Neurobehavioral Status Exam'),
 (20, 'Notes'),
 (21, 'NIH Toolbox ID'),
 (22, 'NIH Toolbox Password'),
 (23, 'Crystallized Composite fully adju

In [41]:
lancet_id = set(lancet[0][1:])

In [42]:
#lancet dataset accounts for all respondents 
[no_lance for no_lance in tech_id if no_lance not in lancet_id]

[]

In [43]:
lancet.index

RangeIndex(start=0, stop=635, step=1)

In [44]:
#indices in lancet dataset for which record_id responded to either/both prompts
lancet_all_ix = [ix for ix in lancet.index if lancet.loc[ix][0] in tech_id]

In [45]:
lancet_id_ix = lancet.iloc[lancet_all_ix,[0,17]]

In [46]:
#each participant has one diagnosis row, other rows are na --> retain one diagnosis per patient 
lancet_id_ix = lancet_id_ix.dropna()

In [47]:
#dataset breakdown 
lancet_id_ix[17].value_counts()

NL                    34
SCI                    9
MCI                    4
Preclinical AD         2
Preclinical PD/DLB     1
Name: 17, dtype: int64

In [48]:
len(lancet_id_ix) == len(ix_id_series)

True

## Metadata

Gender

In [49]:
len(np.where(lancet.iloc[lancet_all_ix][2] == 'Female')[0])

35

In [50]:
len(np.where(lancet.iloc[lancet_all_ix][2] == 'Male')[0])

15

Age

In [51]:
lancet.iloc[lancet_all_ix][5].describe()

count     50
unique    30
top       58
freq       3
Name: 5, dtype: int64

In [52]:
mean(lancet.iloc[lancet_all_ix][5].dropna())

58.22

In [53]:
max(lancet.iloc[lancet_all_ix][5].dropna())

78

In [54]:
min(lancet.iloc[lancet_all_ix][5].dropna())

27

In [55]:
lancet.iloc[520:530]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,...,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402
520,559,Baseline,Female,Memory Loss Prevention,NaN,72,Yes,No,No,NaN,Seifan,No,New patient Visit,NaN,NaN,Complete,SCI,SCI,4,2016-08-01 00:00:00,NaN,32181,yet,124.37,132.754,135.907,2136,110.115,116.308,2304,5,13,15,13,15,1,>75,NaN,NaN,NaN,NaN,NaN,NaN,107.25,114.31,7.61,30,113.13,119.16,8.64,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1424.8,NaN,NaN,NaN,NaN,94.14,0.716761,2137.3,27.4323,NaN,Complete
521,559,FU Visit 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-02-06 00:00:00,NaN,97232,rrz,132.55,137.606,145.452,2220,112.549,118.316,2336,4,7,7,6,13,4,50-75,NaN,NaN,NaN,NaN,NaN,NaN,113.69,120.02,8.04,29,114.11,120.13,8.69,...,100.2,14,91.4,13.91,90.9,27.9,3.64,4.21,34.4,10.96,10.87,2.25,2.6,20.9,6.66,6.64,1.39,1.61,13.4,4.3,4.23,0.39,0.383,0.38,0.39,0.393,0.39,2.6,113.6,2.2,96.8,19.2,151.6,4.9,85.9,4.9,86.3,-5.3,6.8,1314,level 8,81.8,11.4,73.4,NaN,NaN,NaN,NaN,NaN,Complete
522,559,FU Visit 3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-08-28 00:00:00,NaN,64007,qdw,134.24,137.698,145.501,2222,114.952,120.638,2378,6,9,12,10,15,2,>75,NaN,NaN,NaN,NaN,NaN,NaN,108.48,115.25,7.68,28,114.36,120.15,8.7,...,97.6,14.31,93.4,14.31,93.3,28.6,3.48,4.08,33.5,11.2,11.2,2.14,2.54,20.3,6.77,6.77,1.34,1.54,13.2,4.43,4.43,0.392,0.384,0.381,0.391,0.396,0.396,2.6,116.9,2.2,99.5,18.7,149.9,5.3,91.2,5.3,91.2,-5.7,6.2,1313,level 8,80.7,11.4,73.5,NaN,NaN,NaN,NaN,NaN,Complete
523,559,FU Visit 4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-02-06 00:00:00,NaN,47397,unn,123.28,132.978,136.117,2138,108.504,114.961,2280,5,7,8,6,13,2,50-75,NaN,NaN,NaN,NaN,NaN,NaN,111.84,118.3,7.9,29,109.29,115.34,8.45,...,97.7,13.96,90.6,13.82,89.7,27.8,3.57,4.03,33.7,10.94,10.82,2.2,2.49,20.5,6.64,6.57,1.37,1.54,13.2,4.3,4.26,0.391,0.384,0.38,0.391,0.394,0.394,2.9,126.2,2.4,110.6,20.5,164.3,5.3,94.1,5.3,93.8,-8.4,7.5,1300,NaN,87.1,NaN,73.5,NaN,NaN,NaN,NaN,NaN,Complete
524,567,Baseline,Female,Memory Loss Prevention,NaN,27,Yes,No,No,NaN,Isaacson,No,New patient Visit,NaN,NaN,Complete,NL,NL,0,2016-01-29 00:00:00,"logical memory scored out of 50, not included",93701,saw,147.07,142.36,149.182,2190,127.502,131.71,2533,8,10,12,10,14,1,>75,NaN,NaN,NaN,NaN,NaN,NaN,101.39,108.38,9.18,30,96.7,101.52,9.09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1412.27,NaN,NaN,NaN,NaN,92.69,NaN,2118.4,NaN,NaN,Complete
525,567,FU Visit 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-07-21 00:00:00,NaN,54570,kre,149.67,145.368,147.899,2179,129.998,131.635,2529,8,12,13,13,14,0,>75,NaN,NaN,NaN,NaN,NaN,NaN,108.8,107.22,9.11,29,102.7,101.07,9.06,...,95.7,15.7,99.3,15.92,100.7,31.6,3.73,3.7,33.7,12.21,12.41,2.34,2.29,20.9,7.56,7.67,1.39,1.41,12.8,4.65,4.74,0.381,0.375,0.378,0.381,0.381,0.383,3.5,158.7,3.5,161.5,24.9,201.9,7.9,140.5,7.9,140.5,-20.1,1.8,1344,level 10,103.5,11.9,73.1,NaN,NaN,NaN,NaN,NaN,Complete
526,568,Baseline,Male,Memory Loss Prevention,NaN,61,Yes,No,No,NaN,Isaacson,No,Follow-Up Visit,2016-01-11 00:00:00,2016-03-30 00:00:00,Complete,NL,NL,9,2016-01-21 00:00:00,NaN,70423,ter,132.22,129.528,134.212,2128,121.586,128.817,2489,4,6,7,3,10,5,<25,NaN,NaN,NaN,NaN,NaN,NaN,113.86,120.58,9.2,29,105.35,114.1,8.93,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1450.28,NaN,NaN,NaN,NaN,96.74,0.653152,2465.48,31.2367,NaN,Comp

Diagnosis

In [56]:
lancet.iloc[lancet_all_ix][[16,17]].dropna()

,16,17
7,MCI-NOS,MCI
34,Preclinical PD/DLB,Preclinical PD/DLB
63,SCI,SCI
70,NL,NL
101,SCI,SCI
112,NL,NL
125,NL,NL
133,NL,NL
146,NL,NL
192,NL,NL


In [57]:
np.where(min(lancet.iloc[lancet_all_ix][5].dropna()))

(array([0]),)

# Reference

## Column names

In [58]:
datasheet_col_ix_name = [(i,test) for( i,test) in enumerate(datasheet.iloc[0])]

## 466k Word list

In [59]:
#https://raw.githubusercontent.com/dwyl/english-words/master/words.txt
with open('words.txt','r') as words:
    word_list = words.read().splitlines()

word_list = [w.lower() for w in word_list]

In [60]:
len(word_list)

466544

## nltk pos data dictionary

In [61]:
nltk.help.upenn_tagset()

$: dollar
    $ -$ --$ A$ C$ HK$ M$ NZ$ S$ U.S.$ US$
'': closing quotation mark
    ' ''
(: opening parenthesis
    ( [ {
): closing parenthesis
    ) ] }
,: comma
    ,
--: dash
    --
.: sentence terminator
    . ! ?
:: colon or ellipsis
    : ; ...
CC: conjunction, coordinating
    & 'n and both but either et for less minus neither nor or plus so
    therefore times v. versus vs. whether yet
CD: numeral, cardinal
    mid-1890 nine-thirty forty-two one-tenth ten million 0.5 one forty-
    seven 1987 twenty '79 zero two 78-degrees eighty-four IX '60s .025
    fifteen 271,124 dozen quintillion DM2,000 ...
DT: determiner
    all an another any both del each either every half la many much nary
    neither no some such that the them these this those
EX: existential there
    there
FW: foreign word
    gemeinschaft hund ich jeux habeas Haementeria Herr K'ang-si vous
    lutihaw alai je jour objets salutaris fille quibusdam pas trop Monte
    terram fiche oui corporis ...
IN: preposition or

# Feature Extraction

In [62]:
#memory response
mem_corpus = datasheet.iloc[memory_ix,2]

In [63]:
#tech response
tech_corpus = datasheet.iloc[tech_ix,3]

# Helper Functions

### try-catch comprehension hack
https://stackoverflow.com/questions/1528237/how-can-i-handle-exceptions-in-a-list-comprehension-in-python

In [172]:
def catch(func, handle=lambda e : e, *args, **kwargs):
    try:
        return func(*args, **kwargs)
    
    except ZeroDivisionError as z:
        return None
    except statistics.StatisticsError:
        print('Stat error at ix')
        return handle(e)
    except Exception as e:
        print(handle(e))
        return None
        #return handle(e)

## Tokenization

In [65]:
#Mis-spellings, word length
#rm puctuation
#keep contractions, posessives
def list_tokenize(response):
    return re.sub('['+string.punctuation.replace('\'','')+']',' ',response).split()

## Information extraction nltk

In [66]:
#information-extraction
#https://www.nltk.org/book/ch07.html
def ie_preprocess(document):
    #sentence segmentation
    sentences = nltk.sent_tokenize(document) 
    #tokenization
    sentences = [nltk.word_tokenize(sent) for sent in sentences]
    #pos tagging
    sentences = [nltk.pos_tag(sent) for sent in sentences] 
    
    return sentences

## Uniq, V

In [67]:
def uniq_words (response,token_list=None, counts_vector_return=False,uniq_set_return=False):
    if token_list is None: token_list = list_tokenize(response)
    if counts_vector_return and uniq_set_return: return np.unique(token_list,return_counts=True)
    
    if counts_vector_return and not uniq_set_return: return np.unique(token_list,return_counts=True)[1]
    
    if uniq_set_return and not counts_vector_return: return np.unique(token_list)
    #typical case, investigating number of uniq responses
    
    return len(np.unique(token_list))
        

## V1 

In [68]:
def single_appearances (response,counts=None):
    if counts is None: counts = uniq_words(response,counts_vector_return=True)
    return len(np.where(counts==1)[0])

## N-grams

In [69]:
#http://locallyoptimal.com/blog/2013/01/20/elegant-n-gram-generation-in-python/
#input
def find_bigrams(response, return_list=False):
    input_list = list_tokenize(response)
    bigrams = zip(input_list, input_list[1:])
    if return_list: return [b for b in bigrams]
    else: return bigrams

## Word count

In [70]:
def word_count(response):
    return len(list_tokenize(response))

## Sentence count

In [71]:
def sentence_count(response):
    return len(ie_preprocess(response))

### POS tagging

In [72]:
#information-extraction
#https://www.nltk.org/book/ch07.html
def ie_preprocess(document):
    #sentence segmentation
    sentences = nltk.sent_tokenize(document) 
    #tokenization
    sentences = [nltk.word_tokenize(sent) for sent in sentences]
    #pos tagging
    sentences = [nltk.pos_tag(sent) for sent in sentences] 
    
    return sentences

In [334]:
nltk.help.upenn_tagset()

$: dollar
    $ -$ --$ A$ C$ HK$ M$ NZ$ S$ U.S.$ US$
'': closing quotation mark
    ' ''
(: opening parenthesis
    ( [ {
): closing parenthesis
    ) ] }
,: comma
    ,
--: dash
    --
.: sentence terminator
    . ! ?
:: colon or ellipsis
    : ; ...
CC: conjunction, coordinating
    & 'n and both but either et for less minus neither nor or plus so
    therefore times v. versus vs. whether yet
CD: numeral, cardinal
    mid-1890 nine-thirty forty-two one-tenth ten million 0.5 one forty-
    seven 1987 twenty '79 zero two 78-degrees eighty-four IX '60s .025
    fifteen 271,124 dozen quintillion DM2,000 ...
DT: determiner
    all an another any both del each either every half la many much nary
    neither no some such that the them these this those
EX: existential there
    there
FW: foreign word
    gemeinschaft hund ich jeux habeas Haementeria Herr K'ang-si vous
    lutihaw alai je jour objets salutaris fille quibusdam pas trop Monte
    terram fiche oui corporis ...
IN: preposition or

## Rates

In [338]:
#this method is error-prone
def pos_rate(response,pos_response=None,tag_set=None,denom_set=None,N=None,nn=False,pn=False,vb=False,cloc=False):
    
    if pos_response is None: pos_response = ie_preprocess(response)
        
    
    if tag_set is None:
        #common nouns, excluding proper nouns
        if nn: tag_set = ['NN','NNS']
        elif vb: tag_set = ['VB','VBD','VBG','VBN','VBP','VBZ'] 
        elif pn: tag_set = ['PRP','PRP$']
            #engineer: include all specific nouns, unspecified nouns 'stuff' 'things' 'this' in denom -- list better than intuition?
            #consider removing self-reference pronouns or count separately?
        elif cloc: 
            tag_set = ['PRP','PRP$','WP', 'DT','WDT','WP$']
            #amend-?
#             thing_set = ['thing','things','stuff','anything','everything','something']
            denom_set = ['NN','NNP','NNPS','NNS']
#         elif: lexical_density = [] n adj vb advb 
        else: 
            print('Usage: set pos rate metric to True')
            return None
        
    pos_count = sum([1 for sentence in pos_response for word in sentence if word[1] in tag_set])
    
    if denom_set: 
#         print('Thing_set',[word[0] for sentence in pos_response for word in sentence if word[0] in thing_set])
#         print('Pronouns',[word[0] for sentence in pos_response for word in sentence if word[1] in tag_set])
#         pos_count += sum([1 for sentence in pos_response for word in sentence if word[0] in thing_set])
        denom_count = sum([1 for sentence in pos_response for word in sentence if word[1] in denom_set])
        if denom_count == 0: return None
    
    else:
        if N is None: N = word_count(response)
        denom_count = N
        
    return pos_count/denom_count
        

#         sum([1 for sentence in pos_test for word in sentence if word[1] in ['VB','VBD','VBG','VBN','VBP','VBZ'] ])

In [74]:
datasheet.iloc[522,3]

"I'm glad I had my children fairly late, because I'm still very connected to current trends, music, new technology and sources of information. When I started college as a pre-Med bio major at Georgetown, calculators were $135 and because not everyone could afford them, we were required to use slide rules. When I look back on this I can't believe what I was actually able to do with this device-logs, natural logs, all kinds of graphs, etc. I never used a calculator for Calculus problems or fir Chemistry. Research during college, Med school and during my research fellowship was done through reviewing charts by hand, hundreds of them. Even typing manuscripts with footnotes etccwas so tedious and time consuming. I can't believe what time wasters these activities were. It's great that now so much information and data is readily accessible. Being able to use the internet for research and reading is so much easier, faster, more portable. I always check facts and look up any questions I have im

## Function-word rate

In [76]:
from nltk.corpus import stopwords
stopwords.ensure_loaded()
swords = stopwords.words()

def func_rate(response, words=swords):
    response = list_tokenize(response)
    N = len(response)
#     print([word for word in response if word in swords])
    func_count = sum([1 for word in response if word in swords])
    return func_count/N

In [77]:
func_rate(datasheet.iloc[memory_ix[8],2])

0.34210526315789475

## Honore's R 
![image.png](attachment:image.png)
https://books.google.com/books?id=CwC4CwAAQBAJ&pg=PA111&lpg=PA111&dq=honore+statistic+lexical+richness&source=bl&ots=LPnM2j9oX5&sig=HGObkoMYRy6lLsgc80Y7tUMT7vk&hl=en&sa=X&ved=2ahUKEwjv0ubE7vTdAhXwYd8KHVRfCiMQ6AEwCXoECAUQAQ#v=onepage&q=honore&f=false

http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.2.1359&rep=rep1&type=pdf

http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.484.4648&rep=rep1&type=pdf

viii) Honoré’s R statistic.
If V1 denotes the number of hapax legomena recorded, then Honoré (1979) defined his
statistic as:

*R = 100log(N)/(1 - V1/V)*

As a measure of vocabulary richness which has the virtue of being insensitive to text
length, it has been used in stylometric studies by Holmes (1992) and Holmes and
Stylometric analysis of conversational speech
Forsyth (1995). Values of R typically range from 1000 to 2000 with higher values
implying richer vocabularies in the sense that a greater number of words appear
infrequently.


https://pdfs.semanticscholar.org/11f9/ef33ad001f7638ba29ee8109077de92eb1bb.pdf


Assuming log == base e...fits with 1000-2000 assumption




In [333]:
def honore_r(response,v=None,v1=None, N=None):
    if N is None: N = word_count(response)
    if v is None: v = uniq_words(response)
    if v1 is None: 
        counts = uniq_words(response,counts_vector_return=True)
        v1 = len(np.where(counts==1)[0])
    return (100*math.log(N))/(1-(v1/v))

In [165]:
# mem_honore = pd.Series([catch(lambda:honore_r(response)) for response in datasheet.iloc[memory_ix,2]])

In [166]:
# tech_honore = pd.Series([catch(lambda:honore_r(response)) for response in datasheet.iloc[tech_ix,3]])

In [234]:
mem_honore

0      1192.885028
1      2591.567675
2      2329.238717
3      2040.983470
4      2138.333060
5      1991.233676
6      2661.171885
7      1929.526230
8      2328.055142
9      2294.187431
10     2272.793124
11     1804.930944
12     1887.907158
13     5299.990162
14    12277.120903
15     2046.091453
16     2153.941587
17     1826.868628
18     1745.844582
19     1705.152022
20             NaN
21     1627.663455
22     1777.060503
23     2838.080151
24     1549.932818
25             NaN
26     1872.217608
27     8569.175852
28     2183.660552
29     2509.428950
30     4533.362519
31     1915.660222
32     2235.009679
33     8569.175852
34     2333.684973
35     2733.397315
36     1863.125156
37     2376.949594
38             NaN
39     1552.858207
40     2600.028432
41     2724.221033
42     3234.123985
43     2308.987690
44     2027.891770
45     1445.047512
dtype: float64

## Brunet's W
W = N**(V −0.165)

(vii) Brunet’s W index.
This index, devised by Brunet (1978) and used successfully by Holmes and Forsyth
(1995), is a measure of vocabulary richness which is insensitive to text length. It is
defined as:

W= N^(V-0.165)
where N is the text length, V the number of different words and (-0.165) is a scaling
constant proposed by Brunet. The measure generally varies between 10 and 20 with a
low value indicating a lexically richer speech.
https://pdfs.semanticscholar.org/11f9/ef33ad001f7638ba29ee8109077de92eb1bb.pdf

In [169]:
def brunet_w(response,N=None, v=None):
    if N is None: N = word_count(response)
    if v is None: v = uniq_words(response)
    return N**(v-0.165)

## Moving window

In [80]:
#10-window size generator
#https://docs.python.org/release/2.3.5/lib/itertools-example.html
def window(seq, n=10):
    "Returns a sliding window (of width n) over data from the iterable"
    "   s -> (s0,s1,...s[n-1]), (s1,s2,...,sn), ...                   "
    it = iter(seq)
    result = tuple(islice(it, n))
    if len(result) == n:
        yield result    
    for elem in it:
        result = result[1:] + (elem,)
        yield result

In [81]:
[w for w in window(list_tokenize(datasheet.iloc[496,2]),8)]

[('This', 'is', 'too', 'much', 'pressure', 'for', 'a', 'writer')]

|--------------------------------------------------------------------------------------------------------------------------------------------------------------------------|

### Indices to pull other target-options in clinic dataset

In [82]:
#indices to grab latest cog scores
ix_score_latest = [max(datasheet[datasheet[0]==ix].index) for ix in tech_id]

In [83]:
ix_score_latest_completed = []
for ix in tech_id:
    slice_ix = list(datasheet[datasheet[0]==ix].index)
    while datasheet.iloc[max(slice_ix)][96] != 'Complete':
        slice_ix.remove(max(slice_ix))
    ix_score_latest_completed += [max(slice_ix)]

# Held-out test set

np.random.choice(tech_ix,(1,int(.20*len(tech_ix))),replace=False)

array([[ 29, 464, 324, 277, 242, 496, 107, 359, 411, 375]], dtype=int64)

In [84]:
held_out_test_ix = np.array([ 29, 464, 324, 277, 242, 496, 107, 359, 411, 375])

In [85]:
held_out_test_id = [i for i in datasheet.iloc[held_out_test_ix][0]]

In [86]:
#held out test-set coincidently has both tech + mem responses
#dev set has one tech_x responose
tech_x_ix

[234, 280, 337, 519]

In [87]:
y_held_out_test_ix = [yt for yt in ix_score_latest_completed if datasheet.iloc[yt,0] in held_out_test_id]

np.random.choice(tech_ix_train_dev,(1,int(.20*len(tech_ix_train_dev))),replace=False)

## Dev set

In [88]:
train_dev_ix = [t for t in tech_ix if t not in held_out_test_ix]

tech_ix_dev = np.random.choice(tech_ix_train_dev)

In [89]:
dev_ix = np.array([460,  78, 234, 352, 347, 398, 448, 537])

In [90]:
dev_id = [i for i in datasheet.iloc[dev_ix][0]]

In [91]:
y_dev_ix = [yd for yd in ix_score_latest_completed if datasheet.iloc[yd,0] in dev_id]

## Train set

In [92]:
train_ix = [i for i in train_dev_ix if i not in dev_ix]

In [93]:
train_id = [i for i in datasheet.iloc[train_ix][0]]

In [94]:
len(train_ix)

32

In [95]:
train_ix_mem = [m for m in train_ix if m in memory_ix]

In [96]:
len(train_ix_mem)

29

In [97]:
y_train_ix = [yr for yr in ix_score_latest_completed if datasheet.iloc[yr,0] in train_id]

# Features

## Response length

### Response word count

In [98]:
mem_n = pd.Series([word_count(r) for r in datasheet.iloc[memory_ix,2]],index=memory_ix)

In [99]:
mean_mem_length = mean(mem_n)

In [100]:
mean_mem_length

107.95652173913044

In [101]:
tech_x_proxy_length = pd.Series([mean_mem_length for t in tech_x_ix],index=tech_x_ix)

In [102]:
tech_x_proxy_length

234    107.956522
280    107.956522
337    107.956522
519    107.956522
dtype: float64

mem_n = mem_n.append(tech_x_proxy_length)

#tech ix contains all respondent indices
len(mem_n) == len(tech_ix)

In [103]:
tech_n = pd.Series([word_count(tr) for tr in datasheet.iloc[tech_ix,3]],index=tech_ix)

### Response sentence count

In [104]:
mem_sentence = pd.Series([sentence_count(mr) for mr in datasheet.iloc[memory_ix,2]],index=memory_ix)

mean_mem_sentence = mean(mem_sentence)

tech_x_proxy_sentence = pd.Series([mean_mem_sentence for t in tech_x_ix],index=tech_x_ix)

mem_sentence = mem_sentence.append(tech_x_proxy_sentence)

len(mem_sentence) == len(tech_ix)

In [105]:
tech_sentence = pd.Series([sentence_count(tr) for tr in datasheet.iloc[tech_ix,3]],index=tech_ix)

In [106]:
#And here you have it
#Merged response lengths
#sentences_count = pd.Series([mem_sentence[i]+tech_sentence[i] for i in tech_ix],index=tech_ix)

In [107]:
#sentences_count.describe()

## Lexical Richness

## V

In [108]:
mem_v = pd.Series([uniq_words(mr) for mr in datasheet.iloc[memory_ix,2]],index=memory_ix)

In [109]:
tech_v = pd.Series([uniq_words(tr) for tr in datasheet.iloc[tech_ix,3]],index=tech_ix)

## V1

In [110]:
mem_v1 = pd.Series([single_appearances(mr) for mr in datasheet.iloc[memory_ix,2]],index=memory_ix)

In [111]:
tech_v1 = pd.Series([single_appearances(tr) for tr in datasheet.iloc[tech_ix,3]],index=tech_ix)

## MATTR (window size 10) 

In [112]:
window_size = 10

In [113]:
import statistics

In [114]:
[w for w in window(list_tokenize(datasheet.iloc[496,2]))]

[]

In [115]:
list_tokenize(datasheet.iloc[496,2])

['This', 'is', 'too', 'much', 'pressure', 'for', 'a', 'writer']

In [116]:
# mem_mattr = []
# for ix,mr in zip(memory_ix,datasheet.iloc[memory_ix,2]):
#     stripped_mr = list_tokenize(mr)
#     if len(stripped_mr) < 10: window_size = len(stripped_mr)
#     else: window_size = 10
        
#     try:
#         mem_mattr.append(mean([len(np.unique(mw))/window_size for mw in window(list_tokenize(mr),window_size)]))
#     except statistics.StatisticsError:
#         print('Stat error at ix',ix, mr)
# mem_mattr_a = pd.Series(mem_mattr,index=memory_ix)

In [117]:
#ratio uniq words in each 10-word window
#avg for all ratios
mem_mattr = pd.Series([mean([len(np.unique(mw))/min(len(list_tokenize(mr)),10) for mw in window(list_tokenize(mr),n=min(len(list_tokenize(mr)),10))]) for mr in datasheet.iloc[memory_ix,2]],index=memory_ix)

In [118]:
# mem_mattr_a.equals(mem_mattr)

In [119]:
tech_mattr = pd.Series([mean([len(np.unique(tw))/min(len(list_tokenize(tr)),10) for tw in window(list_tokenize(tr),n=min(len(list_tokenize(tr)),10))]) for tr in datasheet.iloc[tech_ix,3]],index=tech_ix)

## Rates

Noun

In [120]:
mem_rate_nn = pd.Series([pos_rate(mr,nn=True) for mr in datasheet.iloc[memory_ix,2]],index=memory_ix)

In [121]:
tech_rate_nn = pd.Series([pos_rate(tr,nn=True) for tr in datasheet.iloc[tech_ix,3]],index=tech_ix)

Verb

In [122]:
mem_rate_vb = pd.Series([pos_rate(mr,vb=True) for mr in datasheet.iloc[memory_ix,2]],index=memory_ix)

In [123]:
tech_rate_vb = pd.Series([pos_rate(tr,vb=True) for tr in datasheet.iloc[tech_ix,3]],index=tech_ix)

Pronoun

In [124]:
mem_rate_pn = pd.Series([pos_rate(mr,pn=True) for mr in datasheet.iloc[memory_ix,2]],index=memory_ix)

In [125]:
tech_rate_pn = pd.Series([pos_rate(tr,pn=True) for tr in datasheet.iloc[tech_ix,3]],index=tech_ix) 

## Initiating with Coordinating Conjunction
Counts, 0/1, rate-per-sentences?

In [126]:
def coord_conjunctions_init(response):
    return sum([1 for sentence in ie_preprocess(response) if sentence[0][1] == 'CC'])

In [127]:
mem_cc_init = pd.Series([catch(lambda:coord_conjunctions_init(mr)) for mr in datasheet.iloc[memory_ix,2]],index=memory_ix)

In [128]:
tech_cc_init = pd.Series([catch(lambda:coord_conjunctions_init(tr)) for tr in datasheet.iloc[tech_ix,3]],index=tech_ix)

## Misspellings, words not found in dictionary

In [129]:
def misspelt(response):
    return sum([1 for word in list_tokenize(response) if word.lower() not in word_list])

In [130]:
mem_misspell = pd.Series([catch(lambda:misspelt(mr)) for mr in datasheet.iloc[memory_ix,2]],index=memory_ix)

In [131]:
tech_misspell = pd.Series([catch(lambda:misspelt(tr)) for tr in datasheet.iloc[tech_ix,3]],index=tech_ix)

## Syntactic complexity

### Height of constituency tree, Stanford CoreNLP + nltk Tree

https://www.khalidalnajjar.com/setup-use-stanford-corenlp-server-python/


In [185]:
'''
A sample code usage of the python package stanfordcorenlp to access a Stanford CoreNLP server.
Written as part of the blog post: https://www.khalidalnajjar.com/how-to-setup-and-use-stanford-corenlp-server-with-python/ 

cd stanford-corenlp-full-2018-10-05/ 
java -mx4g -cp "*" edu.stanford.nlp.pipeline.StanfordCoreNLPServer -annotators "tokenize,ssplit,pos,lemma,parse,sentiment" -port 9000 -timeout 30000
'''

from stanfordcorenlp import StanfordCoreNLP
import logging
import json

class StanfordNLP:
    def __init__(self, host='http://localhost', port=9000):
        self.nlp = StanfordCoreNLP(host, port=port,
                                   timeout=30000)  # , quiet=False, logging_level=logging.DEBUG)
        self.props = {
            'annotators': 'tokenize,ssplit,pos,lemma,ner,parse,depparse,dcoref,relation',
            'pipelineLanguage': 'en',
            'outputFormat': 'json'
        }

    def word_tokenize(self, sentence):
        return self.nlp.word_tokenize(sentence)

    def pos(self, sentence):
        return self.nlp.pos_tag(sentence)

    def ner(self, sentence):
        return self.nlp.ner(sentence)

    def parse(self, sentence):
        return self.nlp.parse(sentence)

    def dependency_parse(self, sentence):
        return self.nlp.dependency_parse(sentence)

    def annotate(self, sentence):
        return json.loads(self.nlp.annotate(sentence, properties=self.props))

    @staticmethod
    def tokens_to_dict(_tokens):
        tokens = defaultdict(dict)
        for token in _tokens:
            tokens[int(token['index'])] = {
                'word': token['word'],
                'lemma': token['lemma'],
                'pos': token['pos'],
                'ner': token['ner']
            }
        return tokens

In [186]:
from nltk.tree import Tree

In [187]:
sNLP = StanfordNLP()

INFO:root:Using an existing server http://localhost:9000
INFO:root:The server is available.


In [188]:
mem_syntree = pd.Series([Tree.fromstring(str(sNLP.parse(mr))).height() for mr in datasheet.iloc[memory_ix,2]],index=memory_ix)

INFO:root:{'properties': "{'annotators': 'pos,parse', 'outputFormat': 'json'}", 'pipelineLanguage': 'en'}
INFO:root:{'properties': "{'annotators': 'pos,parse', 'outputFormat': 'json'}", 'pipelineLanguage': 'en'}
INFO:root:{'properties': "{'annotators': 'pos,parse', 'outputFormat': 'json'}", 'pipelineLanguage': 'en'}
INFO:root:{'properties': "{'annotators': 'pos,parse', 'outputFormat': 'json'}", 'pipelineLanguage': 'en'}
INFO:root:{'properties': "{'annotators': 'pos,parse', 'outputFormat': 'json'}", 'pipelineLanguage': 'en'}
INFO:root:{'properties': "{'annotators': 'pos,parse', 'outputFormat': 'json'}", 'pipelineLanguage': 'en'}
INFO:root:{'properties': "{'annotators': 'pos,parse', 'outputFormat': 'json'}", 'pipelineLanguage': 'en'}
INFO:root:{'properties': "{'annotators': 'pos,parse', 'outputFormat': 'json'}", 'pipelineLanguage': 'en'}
INFO:root:{'properties': "{'annotators': 'pos,parse', 'outputFormat': 'json'}", 'pipelineLanguage': 'en'}
INFO:root:{'properties': "{'annotators': 'pos,

In [189]:
tech_syntree = pd.Series([Tree.fromstring(str(sNLP.parse(tr))).height() for tr in datasheet.iloc[tech_ix,3]],index=tech_ix)

INFO:root:{'properties': "{'annotators': 'pos,parse', 'outputFormat': 'json'}", 'pipelineLanguage': 'en'}
INFO:root:{'properties': "{'annotators': 'pos,parse', 'outputFormat': 'json'}", 'pipelineLanguage': 'en'}
INFO:root:{'properties': "{'annotators': 'pos,parse', 'outputFormat': 'json'}", 'pipelineLanguage': 'en'}
INFO:root:{'properties': "{'annotators': 'pos,parse', 'outputFormat': 'json'}", 'pipelineLanguage': 'en'}
INFO:root:{'properties': "{'annotators': 'pos,parse', 'outputFormat': 'json'}", 'pipelineLanguage': 'en'}
INFO:root:{'properties': "{'annotators': 'pos,parse', 'outputFormat': 'json'}", 'pipelineLanguage': 'en'}
INFO:root:{'properties': "{'annotators': 'pos,parse', 'outputFormat': 'json'}", 'pipelineLanguage': 'en'}
INFO:root:{'properties': "{'annotators': 'pos,parse', 'outputFormat': 'json'}", 'pipelineLanguage': 'en'}
INFO:root:{'properties': "{'annotators': 'pos,parse', 'outputFormat': 'json'}", 'pipelineLanguage': 'en'}
INFO:root:{'properties': "{'annotators': 'pos,

Sentence complexity 

In [ ]:
scipy.stats.pearsonr(mem_syntree,tech_syntree[memory_ix])

In [ ]:
scipy.stats.skew(mem_syntree)

In [ ]:
scipy.stats.skew(tech_syntree)

In [ ]:
scipy.stats.kurtosis(mem_syntree)

In [ ]:
scipy.stats.kurtosis(tech_syntree)

In [ ]:
scipy.stats.spearmanr(mem_syntree,tech_syntree[memory_ix])

https://github.com/Lynten/stanford-corenlp/issues/59

## Reduced sentences

In [ ]:
# Reduced sentences:
#  Following the definition set out by [25], this feature represents those subordinated sentences without a conjunction but with nominal verb forms (which are either participles or gerund). To obtain the count for this feature, the frequencies of POS tags (VBG and VBN) are used.

# From <https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5237556/#CR25> 


In [199]:
mem_reduced = pd.Series([sum([1 for (word,tag) in sNLP.pos(mr) if tag in ['VBG','VBN']]) for mr in datasheet.iloc[memory_ix,2]],index=memory_ix)

INFO:root:{'properties': "{'annotators': 'pos', 'outputFormat': 'json'}", 'pipelineLanguage': 'en'}
INFO:root:{'properties': "{'annotators': 'pos', 'outputFormat': 'json'}", 'pipelineLanguage': 'en'}
INFO:root:{'properties': "{'annotators': 'pos', 'outputFormat': 'json'}", 'pipelineLanguage': 'en'}
INFO:root:{'properties': "{'annotators': 'pos', 'outputFormat': 'json'}", 'pipelineLanguage': 'en'}
INFO:root:{'properties': "{'annotators': 'pos', 'outputFormat': 'json'}", 'pipelineLanguage': 'en'}
INFO:root:{'properties': "{'annotators': 'pos', 'outputFormat': 'json'}", 'pipelineLanguage': 'en'}
INFO:root:{'properties': "{'annotators': 'pos', 'outputFormat': 'json'}", 'pipelineLanguage': 'en'}
INFO:root:{'properties': "{'annotators': 'pos', 'outputFormat': 'json'}", 'pipelineLanguage': 'en'}
INFO:root:{'properties': "{'annotators': 'pos', 'outputFormat': 'json'}", 'pipelineLanguage': 'en'}
INFO:root:{'properties': "{'annotators': 'pos', 'outputFormat': 'json'}", 'pipelineLanguage': 'en'}


In [200]:
tech_reduced = pd.Series([sum([1 for (word,tag) in sNLP.pos(tr) if tag in ['VBG','VBN']]) for tr in datasheet.iloc[tech_ix,3]],index=tech_ix)

INFO:root:{'properties': "{'annotators': 'pos', 'outputFormat': 'json'}", 'pipelineLanguage': 'en'}
INFO:root:{'properties': "{'annotators': 'pos', 'outputFormat': 'json'}", 'pipelineLanguage': 'en'}
INFO:root:{'properties': "{'annotators': 'pos', 'outputFormat': 'json'}", 'pipelineLanguage': 'en'}
INFO:root:{'properties': "{'annotators': 'pos', 'outputFormat': 'json'}", 'pipelineLanguage': 'en'}
INFO:root:{'properties': "{'annotators': 'pos', 'outputFormat': 'json'}", 'pipelineLanguage': 'en'}
INFO:root:{'properties': "{'annotators': 'pos', 'outputFormat': 'json'}", 'pipelineLanguage': 'en'}
INFO:root:{'properties': "{'annotators': 'pos', 'outputFormat': 'json'}", 'pipelineLanguage': 'en'}
INFO:root:{'properties': "{'annotators': 'pos', 'outputFormat': 'json'}", 'pipelineLanguage': 'en'}
INFO:root:{'properties': "{'annotators': 'pos', 'outputFormat': 'json'}", 'pipelineLanguage': 'en'}
INFO:root:{'properties': "{'annotators': 'pos', 'outputFormat': 'json'}", 'pipelineLanguage': 'en'}


In [ ]:
scipy.stats.spearmanr(mem_reduced,tech_reduced[memory_ix])

## Sentiment analysis/Emotion recognition

In [202]:
# http://t-redactyl.io/blog/2017/04/using-vader-to-handle-sentiment-analysis-with-social-media-text.html
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
vader = SentimentIntensityAnalyzer()

In [332]:
vader

In [203]:
for prompt in datasheet.iloc[memory_ix,2]:
    print(prompt)
    print(vader.polarity_scores(prompt))
    print('--------------------------------------\n')

When I was a little child I had the same experience over and over.  It always occurred between being awake and asleep.  The experience had no words...it was all sensation. At a point between my eyes I got a feeling of all space expanding and expanding until it became so vast I was terrified.  Then all space would contract until it became so tiny I would be terrified. I was both afraid of having this experience and welcomed it.
{'neg': 0.135, 'neu': 0.816, 'pos': 0.049, 'compound': -0.8328}
--------------------------------------

When I was very young, living in the Astoria neighborhood of NYC, my parents took my little brother and me to a huge public swimming pool.  It really was big -- not just in a child's eye -- built during the Depression to accommodate thousands of working class New Yorkers.  The water was very chlorinated.  The pool was concrete, bottom painted bright blue, and had a large shallow portion where kids could play.  I remember the heat, the brilliant light, the chill

In [204]:
# One of my fondest childhood memories is how much I loved growing up around so much family.  My father's family all lived near us and we together frequently.  It was always so much happiness when everyone was together.
# {'neg': 0.0, 'neu': 0.782, 'pos': 0.218, 'compound': 0.8583}

In [205]:
# I have a very clear memeory of last time I saw my father before he passed away in an accident. I was 14 years old.   I do not need to explain why this is important for me .
# {'neg': 0.084, 'neu': 0.788, 'pos': 0.128, 'compound': 0.1513}

In [206]:
mem_sent_neg = pd.Series([vader.polarity_scores(prompt)['neg'] for prompt in datasheet.iloc[memory_ix,2]],index=memory_ix)

In [207]:
mem_sent_neu = pd.Series([vader.polarity_scores(prompt)['neu'] for prompt in datasheet.iloc[memory_ix,2]],index=memory_ix)

In [208]:
mem_sent_pos = pd.Series([vader.polarity_scores(prompt)['pos'] for prompt in datasheet.iloc[memory_ix,2]],index=memory_ix)

In [209]:
mem_sent_cpd = pd.Series([vader.polarity_scores(prompt)['compound'] for prompt in datasheet.iloc[memory_ix,2]],index=memory_ix)

In [210]:
tech_sent_neg = pd.Series([vader.polarity_scores(prompt)['neg'] for prompt in datasheet.iloc[tech_ix,3]],index=tech_ix)

In [211]:
tech_sent_neu = pd.Series([vader.polarity_scores(prompt)['neu'] for prompt in datasheet.iloc[tech_ix,3]],index=tech_ix)

In [212]:
tech_sent_pos = pd.Series([vader.polarity_scores(prompt)['pos'] for prompt in datasheet.iloc[tech_ix,3]],index=tech_ix)

In [213]:
tech_sent_cpd = pd.Series([vader.polarity_scores(prompt)['compound'] for prompt in datasheet.iloc[tech_ix,3]],index=tech_ix)

In [214]:
from textblob import TextBlob

In [215]:
for blob in datasheet.iloc[memory_ix,2]:
    print(blob)
    print(TextBlob(blob).sentiment)

When I was a little child I had the same experience over and over.  It always occurred between being awake and asleep.  The experience had no words...it was all sensation. At a point between my eyes I got a feeling of all space expanding and expanding until it became so vast I was terrified.  Then all space would contract until it became so tiny I would be terrified. I was both afraid of having this experience and welcomed it.
Sentiment(polarity=-0.1575, subjectivity=0.605)
When I was very young, living in the Astoria neighborhood of NYC, my parents took my little brother and me to a huge public swimming pool.  It really was big -- not just in a child's eye -- built during the Depression to accommodate thousands of working class New Yorkers.  The water was very chlorinated.  The pool was concrete, bottom painted bright blue, and had a large shallow portion where kids could play.  I remember the heat, the brilliant light, the chill of the water, the happiness in the air.  Swimming and s

In [216]:
mem_sent_pol = pd.Series([TextBlob(datasheet.iloc[mr,2]).sentiment[0] for mr in memory_ix],index=memory_ix)

In [217]:
mem_sent_subj = pd.Series([TextBlob(datasheet.iloc[mr,2]).sentiment[1] for mr in memory_ix],index=memory_ix)

In [218]:
tech_sent_pol = pd.Series([TextBlob(datasheet.iloc[tr,3]).sentiment[0] for tr in tech_ix],index=tech_ix)

In [219]:
tech_sent_subj = pd.Series([TextBlob(datasheet.iloc[tr,3]).sentiment[1] for tr in tech_ix],index=tech_ix)

## Word frequency

## Lexical density - rate (nouns+vb+adj+adv)/n : (gets at semantic anal?)
file:///Users/renee/Downloads/2273-Article%20Text-4927-1-10-20100218.pdf
Lexical density is the term most often used for describing the proportion of
content words (nouns, verbs, adjectives, and often also adverbs) to the total
number of words. By investigating this, we receive a notion of information
packaging; a text with a high proportion of content words contains more
information than a text with a high proportion of function words (prepositions,
interjections, pronouns, conjunctions and count words). 

## Idea density 

https://github.com/andrecunha/idd3/blob/master/run.py

## SEMANTIC (high rate of function words? 

People + object reference words

In [339]:
mem_rate_cloc = pd.Series([pos_rate(mr,cloc=True) for mr in datasheet.iloc[memory_ix,2]],index=memory_ix)

In [340]:
tech_rate_cloc = pd.Series([pos_rate(tr,cloc=True) for tr in datasheet.iloc[tech_ix,3]],index=tech_ix)

### Stopwords

In [249]:
help(stopwords)

Help on WordListCorpusReader in module nltk.corpus.reader.wordlist object:

class WordListCorpusReader(nltk.corpus.reader.api.CorpusReader)
 |  WordListCorpusReader(root, fileids, encoding='utf8', tagset=None)
 |  
 |  List of words, one per line.  Blank lines are ignored.
 |  
 |  Method resolution order:
 |      WordListCorpusReader
 |      nltk.corpus.reader.api.CorpusReader
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  raw(self, fileids=None)
 |  
 |  words(self, fileids=None, ignore_lines_startswith='\n')
 |  
 |  ----------------------------------------------------------------------
 |  Methods inherited from nltk.corpus.reader.api.CorpusReader:
 |  
 |  __init__(self, root, fileids, encoding='utf8', tagset=None)
 |      :type root: PathPointer or str
 |      :param root: A path pointer identifying the root directory for
 |          this corpus.  If a string is specified, then it will be
 |          converted to a ``PathPointer`` automatically.
 |      :param fi

In [196]:
mem_rate_func = pd.Series([func_rate(mr) for mr in datasheet.iloc[memory_ix,2]],index=memory_ix)

In [197]:
tech_rate_func = pd.Series([func_rate(tr) for tr in datasheet.iloc[tech_ix,3]],index=tech_ix)

# Feature Matrix

In [341]:
mem_feature_list = [mem_n,mem_sentence,mem_v,mem_v1,mem_mattr,mem_rate_nn,mem_rate_vb,mem_rate_pn,mem_rate_cloc,mem_cc_init,mem_misspell,mem_syntree,mem_reduced,mem_sent_neg,mem_sent_neu,mem_sent_pos,mem_sent_cpd,mem_sent_pol,mem_sent_subj,mem_rate_func,mem_honore]

In [342]:
mem_feature_df = pd.DataFrame(mem_feature_list).transpose()

In [343]:
tech_feature_list = [tech_n,tech_sentence,tech_v,tech_v1,tech_mattr,tech_rate_nn,tech_rate_vb,tech_rate_pn,tech_rate_cloc,tech_cc_init,tech_misspell,tech_syntree,tech_reduced,tech_sent_neg,tech_sent_neu,tech_sent_pos,tech_sent_cpd,tech_sent_pol,tech_sent_subj,tech_rate_func,tech_honore]

In [344]:
tech_feature_df = pd.DataFrame(tech_feature_list).transpose()

In [345]:
X = mem_feature_df.join(tech_feature_df,how='outer',lsuffix='_mem_features',rsuffix='_tech_features')

Replace nan with col means

In [346]:
X = X.fillna(X.mean())

## Feature Interaction

### Linear: Pearsons, Spearmans - helper function

In [347]:
def pearsons_corr(y,X=X):
    pearsons_sig = []
    for col_name in X.columns:
        p_corr = scipy.stats.pearsonr(X[col_name],y)
        print('pearsons:',col_name,p_corr)
        if p_corr[1] < 0.05:
            pearsons_sig += [(col_name, p_corr)]
    return pearsons_sig

In [348]:
def spearmans_corr(y,X=X):
    spearmans_sig = []
    for col_name in X.columns:
        s_corr = scipy.stats.spearmanr(X[col_name],y)
        print('spearmans:',col_name,s_corr)
        if s_corr[1] < 0.05:
            spearmans_sig += [(col_name,s_corr)]
    return spearmans_sig
        


### Interaction

https://chrisalbon.com/machine_learning/linear_regression/adding_interaction_terms/

### Clustering

### Information Gain

### Mutual Information

### Attribute interaction

### Bonferonni Adjustment

## Feature Correlation - Pearson's R [Linear Relationship] 

Skew

In [349]:
X.columns = ['mem_n','mem_sentence','mem_v','mem_v1','mem_mattr','mem_rate_nn','mem_rate_vb','mem_rate_pn','mem_rate_cloc','mem_cc_init','mem_misspell','mem_syntree','mem_reduced','mem_sent_neg','mem_sent_neu','mem_sent_pos','mem_sent_cpd','mem_sent_pol','mem_sent_subj','mem_rate_func','mem_honore','tech_n','tech_sentence','tech_v','tech_v1','tech_mattr','tech_rate_nn','tech_rate_vb','tech_rate_pn','tech_rate_cloc','tech_cc_init','tech_misspell','tech_syntree','tech_reduced','tech_sent_neg','tech_sent_neu','tech_sent_pos','tech_sent_cpd','tech_sent_pol','tech_sent_subj','tech_rate_func','tech_honore']

In [350]:
for col_name in X.columns:
    print(col_name,scipy.stats.skew(X[col_name]))

mem_n 2.5552181701157406
mem_sentence 3.0261703480268443
mem_v 2.0358939151052833
mem_v1 1.953087963960873
mem_mattr -0.4737875371806025
mem_rate_nn 0.3158556533236132
mem_rate_vb -0.052022188583841854
mem_rate_pn 0.15833383692561173
mem_rate_cloc 1.3798809405821726
mem_cc_init 3.6717340967701584
mem_misspell 1.9770116039033
mem_syntree 1.9788548998248747
mem_reduced 2.6288175118747716
mem_sent_neg 3.185068348325173
mem_sent_neu -0.6497222959921051
mem_sent_pos 1.0748435152964944
mem_sent_cpd -1.1451157892777308
mem_sent_pol 1.258750221686934
mem_sent_subj 0.5195440250766056
mem_rate_func -0.9002272950225094
mem_honore 3.3446813351694606
tech_n 2.5715795006972892
tech_sentence 3.3317369625518887
tech_v 2.240466527565758
tech_v1 2.0933975339554607
tech_mattr -1.6578705065218304
tech_rate_nn -0.3817897327317599
tech_rate_vb -0.13422714040838493
tech_rate_pn -0.00488255480553699
tech_rate_cloc 1.3270478582029268
tech_cc_init 3.3452957866378576
tech_misspell 1.8040920774004798
tech_syntree

In [351]:
for col_name in X.columns:
    print(col_name,scipy.stats.kurtosis(X[col_name]))

mem_n 8.296003519071396
mem_sentence 12.542519224241406
mem_v 5.652739679229223
mem_v1 5.0335667356744445
mem_mattr 0.8276861987747237
mem_rate_nn 0.1283545601623839
mem_rate_vb 0.8887068747197198
mem_rate_pn 0.9068690190104105
mem_rate_cloc 3.1788030006122634
mem_cc_init 11.568587799123694
mem_misspell 4.7733269567043335
mem_syntree 6.930556768394544
mem_reduced 9.915411502635088
mem_sent_neg 12.48294186287374
mem_sent_neu 1.0971457742509019
mem_sent_pos 1.4691010466301497
mem_sent_cpd 0.18583968615250734
mem_sent_pol 2.043772945925677
mem_sent_subj 2.15084053379349
mem_rate_func 1.2389273735460318
mem_honore 11.44268327839514
tech_n 6.625948864886869
tech_sentence 14.556303207428709
tech_v 5.5725160383539905
tech_v1 4.896177435161661
tech_mattr 5.2262099387172505
tech_rate_nn 1.8910724704058728
tech_rate_vb -0.024822646563046113
tech_rate_pn -0.08042431168991016
tech_rate_cloc 2.721609071292227
tech_cc_init 11.05711870547035
tech_misspell 3.1012886278889287
tech_syntree -0.1137028864

In [352]:
X.corr()

,mem_n,mem_sentence,mem_v,mem_v1,mem_mattr,mem_rate_nn,mem_rate_vb,mem_rate_pn,mem_rate_cloc,mem_cc_init,mem_misspell,mem_syntree,mem_reduced,mem_sent_neg,mem_sent_neu,mem_sent_pos,mem_sent_cpd,mem_sent_pol,mem_sent_subj,mem_rate_func,mem_honore,tech_n,tech_sentence,tech_v,tech_v1,tech_mattr,tech_rate_nn,tech_rate_vb,tech_rate_pn,tech_rate_cloc,tech_cc_init,tech_misspell,tech_syntree,tech_reduced,tech_sent_neg,tech_sent_neu,tech_sent_pos,tech_sent_cpd,tech_sent_pol,tech_sent_subj,tech_rate_func,tech_honore
mem_n,1.000000,0.942389,0.992229,0.980716,-0.160368,0.085424,-0.080399,-0.004202,-0.110414,0.226988,0.731728,0.237913,0.811551,-0.037661,0.192825,-0.164370,0.088327,-0.320662,-0.077805,0.175284,-0.228463,0.540773,0.655862,0.573238,0.537121,0.076226,0.028304,-0.066542,-0.024526,-0.077337,0.007944,0.435272,0.313898,0.411480,-0.078460,0.169558,-0.155574,0.207776,-0.133315,0.098715,-0.102057,-0.233335
mem_sentence,0.942389,1.000000,0.927837,0.913189,-0.160084,0.029337,-0.053033,0.077739,-0.045167,0.097653,0.704276,0.101894,0.649168,-0.017614,0.121707,-0.108787,0.110574,-0.265414,-0.044047,0.103074,-0.231775,0.484644,0.700012,0.523580,0.496856,0.105813,0.025862,-0.058963,0.056276,-0.039726,0.027090,0.359399,0.227479,0.335962,-0.091151,0.245652,-0.240892,0.100651,-0.144212,0.025600,-0.141158,-0.187231
mem_v,0.992229,0.927837,1.000000,0.995708,-0.157956,0.104488,-0.111439,-0.026518,-0.147914,0.210504,0.726740,0.266170,0.805795,-0.057378,0.215184,-0.170364,0.092603,-0.349896,-0.071895,0.181621,-0.228785,0.533192,0.633578,0.570871,0.538608,0.070337,0.021308,-0.045914,-0.036811,-0.079386,0.007101,0.436354,0.324608,0.397715,-0.076199,0.172429,-0.160628,0.198070,-0.142527,0.102597,-0.112881,-0.236855
mem_v1,0.980716,0.913189,0.995708,1.000000,-0.130477,0.095279,-0.114167,-0.044163,-0.164114,0.188175,0.725193,0.305979,0.792253,-0.054616,0.208935,-0.166239,0.087053,-0.341596,-0.057423,0.177458,-0.185709,0.535798,0.624764,0.574029,0.543807,0.047071,0.001058,-0.019092,-0.042713,-0.073587,-0.006167,0.440513,0.346708,0.397923,-0.053543,0.151896,-0.150345,0.188459,-0.142986,0.111111,-0.114498,-0.222851
mem_mattr,-0.160368,-0.160084,-0.157956,-0.130477,1.000000,-0.009368,0.214355,-0.134161,-0.043436,0.095764,-0.071912,0.163614,-0.156265,0.339812,-0.167373,-0.117404,-0.162912,-0.094601,-0.083921,0.278056,0.416538,0.073798,0.030269,0.031366,0.026628,-0.375625,-0.348722,0.267766,0.172064,0.196932,0.026698,0.072699,-0.060696,0.072057,0.241001,-0.468735,0.416312,0.134476,0.203766,0.220547,0.159025,0.071317
mem_rate_nn,0.085424,0.029337,0.104488,0.095279,-0.009368,1.000000,-0.480062,-0.545082,-0.660756,0.033341,0.107367,0.052019,0.059573,-0.183290,0.163169,-0.011650,0.315487,-0.043772,0.023873,-0.044445,-0.144190,0.232464,0.260844,0.238202,0.230808,0.047622,0.023358,0.146446,-0.056389,-0.019338,-0.047271,0.226751,0.133639,0.199693,-0.156602,-0.001129,0.102723,0.331958,0.163867,0.151342,-0.182985,-0.036391
mem_rate_vb,-0.080399,-0.053033,-0.111439,-0.114167,0.214355,-0.480062,1.000000,0.411372,0.548865,0.024520,-0.228269,0.183757,0.098310,0.217311,-0.193831,0.013440,-0.232315,0.069500,0.405006,0.119727,0.268267,-0.192757,-0.217613,-0.191275,-0.181826,-0.147294,0.125999,-0.122205,-0.224722,-0.118821,0.116098,-0.249943,-0.000031,-0.150366,0.172609,-0.272185,0.220236,-0.034505,0.183378,-0.030773,-0.079141,0.230660
mem_rate_pn,-0.004202,0.077739,-0.026518,-0.044163,-0.134161,-0.545082,0.411372,1.000000,0.769593,0.059031,-0.131310,-0.243877,-0.023158,0.046444,-0.217250,0.182277,-0.100965,0.024712,-0.153573,-0.232966,-0.045140,-0.263150,-0.155272,-0.230288,-0.218898,0.070826,0.024330,-0.150585,0.169031,0.103277,0.101537,-0.216378,-0.327416,-0.264778,0.140160,0.183172,-0.315224,-0.370666,-0.140113,-0.373027,0.069491,0.115329
mem_rate_cloc,-0.110414,-0.045167,-0.147914,-0.164114,-0.043436,-0.660756,0.548865,0.769593,1.000000,-0.018695,-0.207687,-0.172755,-0.051599,0.457178,-0.407638,0.028199,-0.343763,0.005882,-0.133709,-0.060958,0.069854,-0.120736,-0.150075,

In [353]:
X.shape

(50, 42)

## Target: Diagnosis

## y: clinical diagnosis

In [354]:
lancet_target = [0 if record_id=='NL' else 1 for record_id in lancet_id_ix[17]]

In [355]:
lancet_target = pd.Series(lancet_target,index=lancet_id_ix[0])

In [356]:
#order target by X.index
#ix_id_series indexed by X.index
#lancet_target.loc indexed by ix_id_series record_id
y_diagnosis = [lancet_target.loc[ix_id_series[ix]] for ix in X.index]

In [357]:
#SCI considered cognitive typical, 0
lancet_target_sci_ct = [0 if record_id=='NL' or record_id =='SCI' else 1 for record_id in lancet_id_ix[17]]

In [358]:
lancet_target_sci_ct = pd.Series(lancet_target_sci_ct,index=lancet_id_ix[0])

In [359]:
y_sci_ct = pd.Series([lancet_target_sci_ct.loc[ix_id_series[ix]] for ix in X.index],index=X.index)

In [360]:
p_sig_diagnosis = pearsons_corr(y_sci_ct)

pearsons: mem_n (-0.1620577371876187, 0.2608477915850901)
pearsons: mem_sentence (-0.09896598484709615, 0.4941160591562246)
pearsons: mem_v (-0.17751232151889695, 0.21747263732044436)
pearsons: mem_v1 (-0.18534512769163927, 0.19752391183228693)
pearsons: mem_mattr (0.04875099049248901, 0.7367181160638914)
pearsons: mem_rate_nn (-0.16031017424732724, 0.2660933356900946)
pearsons: mem_rate_vb (0.34744886395245156, 0.013426475542973972)
pearsons: mem_rate_pn (0.10442921294779453, 0.470462917918982)
pearsons: mem_rate_cloc (0.24237448769143854, 0.08990697742197647)
pearsons: mem_cc_init (-0.09523585573813051, 0.510612457140804)
pearsons: mem_misspell (-0.23071272761089295, 0.10697212949611531)
pearsons: mem_syntree (-0.05677539497698763, 0.6953355430107788)
pearsons: mem_reduced (-0.042855918419822565, 0.7676056817716231)
pearsons: mem_sent_neg (0.27146852500700147, 0.056520192404567744)
pearsons: mem_sent_neu (-0.08659454412134633, 0.5498708777596584)
pearsons: mem_sent_pos (-0.1418206209

In [361]:
p_sig_diagnosis

[('mem_rate_vb', (0.34744886395245156, 0.013426475542973972))]

In [362]:
s_sig_diagnosis = spearmans_corr(y_sci_ct)

spearmans: mem_n SpearmanrResult(correlation=-0.16381124094480248, pvalue=0.2556544822188728)
spearmans: mem_sentence SpearmanrResult(correlation=-0.07424735277965823, pvalue=0.6083449634641536)
spearmans: mem_v SpearmanrResult(correlation=-0.16782679813585674, pvalue=0.2440252582896008)
spearmans: mem_v1 SpearmanrResult(correlation=-0.18383239185333836, pvalue=0.2012717601507452)
spearmans: mem_mattr SpearmanrResult(correlation=0.045955888811125105, pvalue=0.7513152359084909)
spearmans: mem_rate_nn SpearmanrResult(correlation=-0.1218625094118087, pvalue=0.3991995022606595)
spearmans: mem_rate_vb SpearmanrResult(correlation=0.33360756137816955, pvalue=0.017905627868565505)
spearmans: mem_rate_pn SpearmanrResult(correlation=0.11987340381459151, pvalue=0.40699695110148415)
spearmans: mem_rate_cloc SpearmanrResult(correlation=0.1600274374560178, pvalue=0.2669485687981431)
spearmans: mem_cc_init SpearmanrResult(correlation=-0.00662605269265806, pvalue=0.9635743246974555)
spearmans: mem_mis

In [363]:
s_sig_diagnosis

[('mem_rate_vb',
  SpearmanrResult(correlation=0.33360756137816955, pvalue=0.017905627868565505)),
 ('tech_n',
  SpearmanrResult(correlation=-0.3156438768371438, pvalue=0.02555481658933852)),
 ('tech_v',
  SpearmanrResult(correlation=-0.2916849404049138, pvalue=0.03985129537548607)),
 ('tech_sent_pos',
  SpearmanrResult(correlation=0.2891379500315891, pvalue=0.04169735103213693))]

### y: MMSE

In [364]:
y_mmse = pd.Series([y for y in datasheet.iloc[ix_score_latest_completed,74]])

In [365]:
y_mmse = y_mmse.fillna(math.floor(y_mmse.mean()))

In [366]:
p_sig_mmse = pearsons_corr(y_mmse)

pearsons: mem_n (0.19960714863552925, 0.16459949500875143)
pearsons: mem_sentence (0.16157633437836616, 0.2622858239453544)
pearsons: mem_v (0.20776903381208256, 0.1476609266748482)
pearsons: mem_v1 (0.21740845306433204, 0.1293594591870927)
pearsons: mem_mattr (-0.21714958694781392, 0.12982752588231145)
pearsons: mem_rate_nn (0.06136169686174484, 0.6720635291765167)
pearsons: mem_rate_vb (-0.1482139772544509, 0.3043213834635267)
pearsons: mem_rate_pn (-0.2750563428463165, 0.05320921557947716)
pearsons: mem_rate_cloc (-0.31401571497675296, 0.026366468869374875)
pearsons: mem_cc_init (0.16155283085834593, 0.2623561684660318)
pearsons: mem_misspell (0.1578647944862927, 0.2735507399595701)
pearsons: mem_syntree (0.047316848652443025, 0.7441966446651509)
pearsons: mem_reduced (0.13958094019635622, 0.3336609946285436)
pearsons: mem_sent_neg (-0.4115387925681297, 0.00298567233528275)
pearsons: mem_sent_neu (0.17686746176258747, 0.2191752881780771)
pearsons: mem_sent_pos (0.16878049432697906, 

In [367]:
p_sig_mmse

[('mem_rate_cloc', (-0.31401571497675296, 0.026366468869374875)),
 ('mem_sent_neg', (-0.4115387925681297, 0.00298567233528275)),
 ('mem_sent_cpd', (0.29270784204176453, 0.03912904871345365)),
 ('tech_mattr', (0.4213570647608418, 0.002308689314733616)),
 ('tech_sent_subj', (0.35606610812715206, 0.011153108214650485))]

In [368]:
s_sig_mmse = spearmans_corr(y_mmse)

spearmans: mem_n SpearmanrResult(correlation=0.14928149909107052, pvalue=0.3008123826887112)
spearmans: mem_sentence SpearmanrResult(correlation=0.12119112956509405, pvalue=0.40182157653370976)
spearmans: mem_v SpearmanrResult(correlation=0.17779699877189198, pvalue=0.21672393000797574)
spearmans: mem_v1 SpearmanrResult(correlation=0.21844270172355532, pvalue=0.1275020475877381)
spearmans: mem_mattr SpearmanrResult(correlation=-0.07568386612942433, pvalue=0.6014056964341242)
spearmans: mem_rate_nn SpearmanrResult(correlation=-0.07664413056494278, pvalue=0.59678639795767)
spearmans: mem_rate_vb SpearmanrResult(correlation=-0.09768390792374007, pvalue=0.49975468061467765)
spearmans: mem_rate_pn SpearmanrResult(correlation=-0.17887699791104944, pvalue=0.21389984733626888)
spearmans: mem_rate_cloc SpearmanrResult(correlation=-0.11167790086880693, pvalue=0.4400382225974385)
spearmans: mem_cc_init SpearmanrResult(correlation=0.08123302904028383, pvalue=0.5749310966463785)
spearmans: mem_miss

In [369]:
s_sig_mmse

[('tech_sent_subj',
  SpearmanrResult(correlation=0.31137417074267454, pvalue=0.027729130811070507))]

## Targets: Language + Verbal Knowledge

### y: Picture vocab

In [370]:
y_pic_vocab = datasheet.iloc[ix_score_latest_completed,9]

In [371]:
y_pic_vocab.describe()

count      50.0000
unique     50.0000
top       125.8372
freq        1.0000
Name: 9, dtype: float64

In [372]:
s_sig_picvocab = spearmans_corr(y_pic_vocab)

spearmans: mem_n SpearmanrResult(correlation=0.28479478776029404, pvalue=0.045006614019152326)
spearmans: mem_sentence SpearmanrResult(correlation=0.2370537076918076, pvalue=0.09740904964548718)
spearmans: mem_v SpearmanrResult(correlation=0.3112990298649721, pvalue=0.027768736943226093)
spearmans: mem_v1 SpearmanrResult(correlation=0.2765032732821365, pvalue=0.05191913267822324)
spearmans: mem_mattr SpearmanrResult(correlation=-0.04074083489383821, pvalue=0.7787783340347951)
spearmans: mem_rate_nn SpearmanrResult(correlation=0.04260736142488058, pvalue=0.7689162774528295)
spearmans: mem_rate_vb SpearmanrResult(correlation=0.024592920950233587, pvalue=0.8653834055553931)
spearmans: mem_rate_pn SpearmanrResult(correlation=-0.10703048955112317, pvalue=0.45941726895213286)
spearmans: mem_rate_cloc SpearmanrResult(correlation=-0.03838204353039073, pvalue=0.7912906687424803)
spearmans: mem_cc_init SpearmanrResult(correlation=0.11200333352720943, pvalue=0.438698455862114)
spearmans: mem_miss

In [373]:
s_sig_picvocab

[('mem_n',
  SpearmanrResult(correlation=0.28479478776029404, pvalue=0.045006614019152326)),
 ('mem_v',
  SpearmanrResult(correlation=0.3112990298649721, pvalue=0.027768736943226093))]

In [374]:
p_sig_picvocab = pearsons_corr(y_pic_vocab)

pearsons: mem_n (0.20322364536121695, 0.15692726469139315)
pearsons: mem_sentence (0.1716056013365977, 0.23341515801930446)
pearsons: mem_v (0.22803181378356627, 0.11122552631432524)
pearsons: mem_v1 (0.22730814025836504, 0.11239552921503122)
pearsons: mem_mattr (-0.08197382489052979, 0.5714376169798752)
pearsons: mem_rate_nn (0.07209323826412331, 0.6188148561493363)
pearsons: mem_rate_vb (-0.005905647787423143, 0.967532389930601)
pearsons: mem_rate_pn (-0.13999556302554714, 0.33221278430189094)
pearsons: mem_rate_cloc (-0.18505925277179522, 0.19822835806274297)
pearsons: mem_cc_init (0.06662384338656686, 0.6457333315307978)
pearsons: mem_misspell (0.16670368714612224, 0.24724095818601927)
pearsons: mem_syntree (0.0704253645834969, 0.6269733343052084)
pearsons: mem_reduced (0.16865952520069782, 0.24165943023505693)
pearsons: mem_sent_neg (-0.12345637427682452, 0.3930147372539138)
pearsons: mem_sent_neu (0.07914055824003052, 0.5848513158454975)
pearsons: mem_sent_pos (0.0239503471516643

In [375]:
p_sig_picvocab

[]

https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html

### y: Oral reading

In [376]:
y_oral_reading = datasheet.iloc[ix_score_latest_completed,12]

In [377]:
p_sig_orreading = pearsons_corr(y_oral_reading)

pearsons: mem_n (0.3897376126206539, 0.005146431503942649)
pearsons: mem_sentence (0.35251648020231013, 0.012045932988495656)
pearsons: mem_v (0.4007825884426259, 0.003923307674140071)
pearsons: mem_v1 (0.39564266388836855, 0.004456422647454456)
pearsons: mem_mattr (-0.11595696273479433, 0.42260423054547946)
pearsons: mem_rate_nn (0.10792576738716192, 0.4556485045751494)
pearsons: mem_rate_vb (-0.07295656564689193, 0.6146095685086286)
pearsons: mem_rate_pn (-0.1443592466550905, 0.317210247069947)
pearsons: mem_rate_cloc (-0.0883915552465167, 0.5415897534542554)
pearsons: mem_cc_init (0.2138334621433175, 0.1359366781387053)
pearsons: mem_misspell (0.2292805027639204, 0.10922861936543721)
pearsons: mem_syntree (0.026493868330720306, 0.855085554328267)
pearsons: mem_reduced (0.292012733465165, 0.03961866555199223)
pearsons: mem_sent_neg (-0.015906108785729924, 0.912698635762082)
pearsons: mem_sent_neu (-0.011953848087696025, 0.9343354407462982)
pearsons: mem_sent_pos (0.02522730118141353,

In [378]:
p_sig_orreading

[('mem_n', (0.3897376126206539, 0.005146431503942649)),
 ('mem_sentence', (0.35251648020231013, 0.012045932988495656)),
 ('mem_v', (0.4007825884426259, 0.003923307674140071)),
 ('mem_v1', (0.39564266388836855, 0.004456422647454456)),
 ('mem_reduced', (0.292012733465165, 0.03961866555199223)),
 ('mem_honore', (-0.3264378329130902, 0.020686314530001068)),
 ('tech_n', (0.4146939117981588, 0.002751150354524437)),
 ('tech_sentence', (0.30988537296296265, 0.02852271109498324)),
 ('tech_v', (0.43419398820815824, 0.0016303439306499798)),
 ('tech_v1', (0.45169774525956463, 0.0009922223548094167)),
 ('tech_syntree', (0.32489864464127943, 0.021328550589634187)),
 ('tech_reduced', (0.33223410704774403, 0.018412269531288696))]

In [379]:
s_sig_orreading = spearmans_corr(y_oral_reading)

spearmans: mem_n SpearmanrResult(correlation=0.38787618673711827, pvalue=0.005382498330108163)
spearmans: mem_sentence SpearmanrResult(correlation=0.37495305566314613, pvalue=0.007298370466351066)
spearmans: mem_v SpearmanrResult(correlation=0.4205419301601799, pvalue=0.002359192746326009)
spearmans: mem_v1 SpearmanrResult(correlation=0.41713317439365927, pvalue=0.0025811594691376215)
spearmans: mem_mattr SpearmanrResult(correlation=-0.13024575872310779, pvalue=0.36730605237467107)
spearmans: mem_rate_nn SpearmanrResult(correlation=0.07954655075490669, pvalue=0.5829205272297884)
spearmans: mem_rate_vb SpearmanrResult(correlation=0.0006244296335020246, pvalue=0.996566151738456)
spearmans: mem_rate_pn SpearmanrResult(correlation=-0.10313934518234356, pvalue=0.47599220170375356)
spearmans: mem_rate_cloc SpearmanrResult(correlation=0.06752930967001075, pvalue=0.6412445645467004)
spearmans: mem_cc_init SpearmanrResult(correlation=0.3181659417551027, pvalue=0.02433892028338066)
spearmans: me

In [380]:
s_sig_orreading

[('mem_n',
  SpearmanrResult(correlation=0.38787618673711827, pvalue=0.005382498330108163)),
 ('mem_sentence',
  SpearmanrResult(correlation=0.37495305566314613, pvalue=0.007298370466351066)),
 ('mem_v',
  SpearmanrResult(correlation=0.4205419301601799, pvalue=0.002359192746326009)),
 ('mem_v1',
  SpearmanrResult(correlation=0.41713317439365927, pvalue=0.0025811594691376215)),
 ('mem_cc_init',
  SpearmanrResult(correlation=0.3181659417551027, pvalue=0.02433892028338066)),
 ('mem_reduced',
  SpearmanrResult(correlation=0.3037628686313218, pvalue=0.03198804001397038)),
 ('tech_n',
  SpearmanrResult(correlation=0.38966281384287627, pvalue=0.005155739866854472)),
 ('tech_v',
  SpearmanrResult(correlation=0.4186962887842688, pvalue=0.0024771782848086094)),
 ('tech_v1',
  SpearmanrResult(correlation=0.4442469954964155, pvalue=0.0012297228215124368)),
 ('tech_syntree',
  SpearmanrResult(correlation=0.2979938324251178, pvalue=0.03556621970529668))]

In [327]:
y_oral_reading.describe()

count      50.0000
unique     50.0000
top       127.6899
freq        1.0000
Name: 12, dtype: float64

In [382]:
mem_rate_cloc

6      1.533333
29     0.777778
51     1.117647
56     0.923077
78     0.681818
89     1.108696
100    0.588235
107    1.075000
118    1.000000
154    1.000000
184    0.906250
202    1.571429
217    0.875000
226    1.000000
242    2.000000
266    1.041667
277    0.684211
286    0.916667
297    1.000000
303    1.230769
324    1.000000
347    0.913043
352    0.833333
359    1.133333
375    1.166667
387    2.500000
398    1.482759
411    0.500000
423    0.346154
433    0.711538
438    0.478261
445    1.500000
448    1.054348
460    0.857143
464    1.080000
470    1.500000
474    0.840909
477    0.555556
496    1.000000
499    1.833333
502    1.000000
506    1.196429
509    0.727273
522    0.826389
537    1.153846
540    0.906250
dtype: float64

In [328]:
y_oral_reading.std()

9.05349993570542

In [329]:
np.mean(y_oral_reading)

115.56636939999999

In [330]:
min(y_oral_reading)

90.76323

In [331]:
max(y_oral_reading)

130.494

## y: Boston

### y: Rey Auditory Verbal Learning Test (list memory)

## |------------------------------------------------Testing---------------------------------------------------------|

## Init modeling

In [ ]:
from sklearn import linear_model

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
#PULL ONLY TRAIN, DEV INDICES FOR TRAINING
lr = linear_model.LogisticRegression()

In [ ]:
np.shape(X.loc[train_ix])

In [ ]:
np.shape(y.loc[train_ix])

In [ ]:
lr.fit(X.loc[train_ix],y.loc[train_ix])

In [ ]:
lr_preds = lr.predict(X.loc[dev_ix])

In [ ]:
lr_sci = linear_model.LogisticRegression()

In [ ]:
lr_sci.fit(X.loc[train_ix],y.loc[train_ix])

In [ ]:
lr_sci_preds = lr_sci.predict(X.loc[dev_ix])

In [ ]:
accuracy_score(y.loc[dev_ix],lr_sci_preds)

In [ ]:
lr_preds

In [ ]:
accuracy_score(y.loc[dev_ix],lr_preds)

In [ ]:
from sklearn.svm import SVC

In [ ]:
svm = SVC()

In [ ]:
svm.fit(X.loc[train_ix],y.loc[train_ix])

In [ ]:
svm_preds = svm.predict(X.loc[dev_ix])

In [ ]:
accuracy_score(y.loc[dev_ix],svm_preds)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rf = RandomForestClassifier()

In [ ]:
rf.fit(X.loc[train_ix],y.loc[train_ix])

In [ ]:
rf_preds = rf.predict(X.loc[dev_ix])

In [ ]:
accuracy_score(y.loc[dev_ix],rf_preds)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=3)

In [ ]:
knn.fit(X.loc[train_ix],y.loc[train_ix])

In [ ]:
knn_preds = knn.predict(X.loc[dev_ix])

In [ ]:
knn_preds

In [ ]:
accuracy_score(knn_preds,y.loc[dev_ix])

In [ ]:
y.loc[dev_ix]

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(y.loc[dev_ix],knn_preds))

In [ ]:
knn_sci = KNeighborsClassifier(n_neighbors=3)

In [ ]:
knn_sci.fit(X.loc[train_ix],y_sci_ct.loc[train_ix])

In [ ]:
knn_sci_preds = knn_sci.predict(X.loc[dev_ix])

In [ ]:
print(classification_report(y_sci_ct.loc[dev_ix],knn_sci_preds))

In [ ]:
y_sci_ct.loc[dev_ix]

In [ ]:
knn_sci_preds

### Stanford parser

In [144]:
from nltk.parse import CoreNLPParser

In [145]:
# Lexical Parser
parser = CoreNLPParser(url='http://localhost:9000')

In [ ]:
from nltk.tag import StanfordNERTagger
stanford_classifier = r'C:\Users\Renee\StanfordParser\stanford-ner-2018-02-27\classifiers\english.all.3class.distsim.crf.ser.gz'
stanford_ner_path = r'C:\Users\Renee\StanfordParser\stanford-ner-2018-02-27\stanford-ner.jar'

In [156]:
from nltk.tag import StanfordNERTagger
stanford_classifier = r'C:\Users\Renee\StanfordParser\stanford-ner-2018-10-17\classifiers\english.all.3class.distsim.crf.ser.gz'
stanford_ner_path = r'..\stanford-english-kbp-corenlp-2018-10-05-models.jar'


In [ ]:
stanford-srparser-2014-10-23-models.jar


LookupError: Could not find stanford-ner.jar jar file at ..\stanford-english-kbp-corenlp-2018-10-05-models.jar

In [ ]:
[st.tag(t) for t in [stripped_test]]

In [ ]:
java -mx4g -cp "*" edu.stanford.nlp.pipeline.StanfordCoreNLPServer 9000 # run server

In [ ]:
# Parse tokenized text.
list(parser.parse('What is the airspeed of an unladen swallow ?'.split()))
[Tree('ROOT', [Tree('SBARQ', [Tree('WHNP', [Tree('WP', ['What'])]), Tree('SQ', [Tree('VBZ', ['is']), Tree('NP', [Tree('NP', [Tree('DT', ['the']), Tree('NN', ['airspeed'])]), Tree('PP', [Tree('IN', ['of']), Tree('NP', [Tree('DT', ['an']), Tree('JJ', ['unladen'])])]), Tree('S', [Tree('VP', [Tree('VB', ['swallow'])])])])]), Tree('.', ['?'])])])]

# Parse raw string.
>>> list(parser.raw_parse('What is the airspeed of an unladen swallow ?'))
[Tree('ROOT', [Tree('SBARQ', [Tree('WHNP', [Tree('WP', ['What'])]), Tree('SQ', [Tree('VBZ', ['is']), Tree('NP', [Tree('NP', [Tree('DT', ['the']), Tree('NN', ['airspeed'])]), Tree('PP', [Tree('IN', ['of']), Tree('NP', [Tree('DT', ['an']), Tree('JJ', ['unladen'])])]), Tree('S', [Tree('VP', [Tree('VB', ['swallow'])])])])]), Tree('.', ['?'])])])]

# Neural Dependency Parser
>>> from nltk.parse.corenlp import CoreNLPDependencyParser
>>> dep_parser = CoreNLPDependencyParser(url='http://localhost:9000')
>>> parses = dep_parser.parse('What is the airspeed of an unladen swallow ?'.split())
>>> [[(governor, dep, dependent) for governor, dep, dependent in parse.triples()] for parse in parses]
[[(('What', 'WP'), 'cop', ('is', 'VBZ')), (('What', 'WP'), 'nsubj', ('airspeed', 'NN')), (('airspeed', 'NN'), 'det', ('the', 'DT')), (('airspeed', 'NN'), 'nmod', ('swallow', 'VB')), (('swallow', 'VB'), 'case', ('of', 'IN')), (('swallow', 'VB'), 'det', ('an', 'DT')), (('swallow', 'VB'), 'amod', ('unladen', 'JJ')), (('What', 'WP'), 'punct', ('?', '.'))]]


# Tokenizer
>>> parser = CoreNLPParser(url='http://localhost:9000')
>>> list(parser.tokenize('What is the airspeed of an unladen swallow?'))
['What', 'is', 'the', 'airspeed', 'of', 'an', 'unladen', 'swallow', '?']

# POS Tagger
>>> pos_tagger = CoreNLPParser(url='http://localhost:9000', tagtype='pos')
>>> list(pos_tagger.tag('What is the airspeed of an unladen swallow ?'.split()))
[('What', 'WP'), ('is', 'VBZ'), ('the', 'DT'), ('airspeed', 'NN'), ('of', 'IN'), ('an', 'DT'), ('unladen', 'JJ'), ('swallow', 'VB'), ('?', '.')]

# NER Tagger
>>> ner_tagger = CoreNLPParser(url='http://localhost:9000', tagtype='ner')
>>> list(ner_tagger.tag(('Rami Eid is studying at Stony Brook University in NY'.split())))
[('Rami', 'PERSON'), ('Eid', 'PERSON'), ('is', 'O'), ('studying', 'O'), ('at', 'O'), ('Stony', 'ORGANIZATION'), ('Brook', 'ORGANIZATION'), ('University', 'ORGANIZATION'), ('in', 'O'), ('NY', 'STATE_OR_PROVINCE')]

### Misspellings- (stemmed) words not in dictionary

In [ ]:
#lacking
# from nltk.corpus import words
# word_list = words.words()

In [ ]:
#https://github.com/dwyl/english-words

with open('words.txt','r') as words:
    word_list = words.read().splitlines()

word_list = [w.lower() for w in word_list]

In [ ]:
len(word_list)

In [ ]:
#as ratio to response length?...leave as total misspellings, length accounted for
test_misspellings = sum([1 for t in stripped_test if t.lower() not in word_list])

In [ ]:
test_misspellings

In [ ]:
'couldn\'t' in word_list

In [ ]:
'Little' in word_list

In [ ]:
'little' in word_list

In [ ]:
test_response

### Honore's R

In [235]:
mem_honore = pd.Series([catch(lambda:honore_r(mem)) for mem in datasheet.iloc[memory_ix,2]],index=memory_ix)

In [236]:
tech_honore = pd.Series([catch(lambda:honore_r(tech)) for tech in datasheet.iloc[tech_ix,3]],index=tech_ix)

### Brunet's W

In [173]:
mem_brunet = pd.Series([catch(lambda:brunet_w(mem)) for mem in datasheet.iloc[memory_ix,2]],index=memory_ix)

(34, 'Result too large')
(34, 'Result too large')
(34, 'Result too large')
(34, 'Result too large')


In [180]:
unidentified_brunet = [i for i in mem_brunet.index if np.isnan(mem_brunet[i])]

In [183]:
[print(prompt) for prompt in datasheet.iloc[unidentified_brunet,2]]

They say traumatic memories tend to remain vivid and for some reason the first memory that comes to mind from my childhood is the time I crashed my bicycle, with my younger sister riding on the seat behind me, and had to run through our suburban streets with my sister, blood streaming down her face, looking for help. I was probably about 9 years old and my sister was 7, and these were the days when no one wore a bicycle helmet and we routinely 'doubled' each other on bicycles, one person standing up and pedaling while the other sat on the seat, holding the pedaler by the waist. We were headed to our piano lesson and I must have taken a bump too fast and we both went flying. We were essentially fine but a small cut on my sister's forehead seemed to us to be bleeding profusely so we ran the rest of the way to our piano teacher's house. He was a strict, taciturn, Israeli man and was not particularly sympathetic to his two students turning up hysterical and bleeding at his door. However, h

[None, None, None, None]

In [174]:
mem_brunet

6       8.657640e+92
29     9.654888e+145
51     9.119621e+201
56     2.350826e+212
78     7.027930e+101
89     9.889380e+284
100     1.443612e+92
107    1.049744e+251
118     1.960698e+50
154     2.142165e+99
184    1.563139e+187
202     3.950512e+54
217     1.193229e+98
226     6.405757e+22
242     1.148432e+53
266    5.323888e+150
277     3.535378e+74
286     1.855262e+55
297    2.013783e+151
303     1.468933e+81
324     1.167454e+19
347    1.118147e+141
352    2.797573e+115
359              NaN
375    1.293050e+114
387     1.167454e+19
398    1.089446e+211
411     9.534169e+36
423     3.833829e+75
433    1.958330e+250
438    6.598945e+117
445    6.107857e+132
448              NaN
460     9.534169e+36
464    1.216693e+182
470     4.930930e+11
474    1.284307e+218
477    3.222838e+154
496     1.190447e+07
499     1.610924e+40
502    1.504302e+135
506              NaN
509     8.958058e+69
522              NaN
537    6.229468e+175
540    5.330159e+137
dtype: float64

In [ ]:
tech

## Word obscurity

## Syntactic complexity, depth of syntactic parse tree

In [ ]:
# from nltk.corpus import treebank

In [ ]:
#stemming
#https://pythonprogramming.net/stemming-nltk-tutorial/
# from nltk.corpus import wordnet

In [ ]:
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize

ps = PorterStemmer()

In [ ]:
# os.environ['CLASSPATH'] = “stanford-pos”
# os.environ['STANFORD_MODELS'] = “stanford-pos”
# from nltk.tag import StanfordPOSTagger
# st = StanfordPOSTagger(‘english-bidirectional-distsim.tagger’) 
# st.tag(sent.split())

In [ ]:
st.tag(["we\'re"])

In [ ]:
st.tag(['we'])

In [ ]:
st.tag(['are'])

In [ ]:
nltk.pos_tag(['we\'re'])

In [ ]:
nltk.pos_tag(['we'])

In [ ]:
nltk.pos_tag(['are'])

In [ ]:
stagged_mem = [st.tag(t) for t in tokenized_mem]

In [ ]:
stagged_mem

In [ ]:
#https://nlpforhackers.io/stemmers-vs-lemmatizers/
#https://pythonprogramming.net/lemmatizing-nltk-tutorial/
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()


In [ ]:
lemmatizer.lemmatize('notawording')

In [ ]:
lemmatizer.lemmatize('occurred')

In [ ]:
ps.stem('occurred')

In [ ]:
ps.stem('notaworking')